# General information

<i>'No BS, just essential pandas!'</i> 

Cheat sheet to data wrangling with Python pandas. The aim is to cover essential tips and tricks for data work flow. Each topic is featured with a minimal working example. The cheat sheet is work in progress, meaning that all suggestions for improvements are welcome!

Tested on pandas version 0.25.0.

Note: Section numbering and links due to restrictions in Github Jupyter notebook support. Use [nbviewer](https://nbviewer.jupyter.org/) instead.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#General-information" data-toc-modified-id="General-information-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>General information</a></span></li><li><span><a href="#Input/output-data" data-toc-modified-id="Input/output-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Input/output data</a></span><ul class="toc-item"><li><span><a href="#Excel-files" data-toc-modified-id="Excel-files-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Excel files</a></span><ul class="toc-item"><li><span><a href="#Import-Excel-file" data-toc-modified-id="Import-Excel-file-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Import Excel file</a></span></li><li><span><a href="#Import-csv" data-toc-modified-id="Import-csv-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Import csv</a></span></li><li><span><a href="#Export-Excel-file" data-toc-modified-id="Export-Excel-file-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Export Excel file</a></span><ul class="toc-item"><li><span><a href="#Quick-and-dirty" data-toc-modified-id="Quick-and-dirty-2.1.3.1"><span class="toc-item-num">2.1.3.1&nbsp;&nbsp;</span>Quick and dirty</a></span></li><li><span><a href="#More-complete-(overwrites-sheets-however!)" data-toc-modified-id="More-complete-(overwrites-sheets-however!)-2.1.3.2"><span class="toc-item-num">2.1.3.2&nbsp;&nbsp;</span>More complete (overwrites sheets however!)</a></span></li><li><span><a href="#More-complete-(does-not-overwrite-sheets)" data-toc-modified-id="More-complete-(does-not-overwrite-sheets)-2.1.3.3"><span class="toc-item-num">2.1.3.3&nbsp;&nbsp;</span>More complete (does not overwrite sheets)</a></span></li></ul></li></ul></li><li><span><a href="#SQL" data-toc-modified-id="SQL-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>SQL</a></span><ul class="toc-item"><li><span><a href="#Import-SQL-table" data-toc-modified-id="Import-SQL-table-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Import SQL table</a></span></li></ul></li></ul></li><li><span><a href="#Basic-DataFrame-operations" data-toc-modified-id="Basic-DataFrame-operations-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Basic DataFrame operations</a></span><ul class="toc-item"><li><span><a href="#DataFrame-initialization" data-toc-modified-id="DataFrame-initialization-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>DataFrame initialization</a></span></li><li><span><a href="#Modifying-DataFrame" data-toc-modified-id="Modifying-DataFrame-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Modifying DataFrame</a></span><ul class="toc-item"><li><span><a href="#Renaming-columns" data-toc-modified-id="Renaming-columns-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Renaming columns</a></span></li><li><span><a href="#Sort-data-frame-by-colum-values" data-toc-modified-id="Sort-data-frame-by-colum-values-3.2.2"><span class="toc-item-num">3.2.2&nbsp;&nbsp;</span>Sort data frame by colum values</a></span></li><li><span><a href="#Sort-data-frame-by-colums,-custom-order" data-toc-modified-id="Sort-data-frame-by-colums,-custom-order-3.2.3"><span class="toc-item-num">3.2.3&nbsp;&nbsp;</span>Sort data frame by colums, custom order</a></span></li><li><span><a href="#Change-DataFrame-column-order" data-toc-modified-id="Change-DataFrame-column-order-3.2.4"><span class="toc-item-num">3.2.4&nbsp;&nbsp;</span>Change DataFrame column order</a></span></li><li><span><a href="#Changing-data-types" data-toc-modified-id="Changing-data-types-3.2.5"><span class="toc-item-num">3.2.5&nbsp;&nbsp;</span>Changing data types</a></span></li><li><span><a href="#Add-row-to-DataFrame" data-toc-modified-id="Add-row-to-DataFrame-3.2.6"><span class="toc-item-num">3.2.6&nbsp;&nbsp;</span>Add row to DataFrame</a></span></li><li><span><a href="#Add-column-to-DataFrame" data-toc-modified-id="Add-column-to-DataFrame-3.2.7"><span class="toc-item-num">3.2.7&nbsp;&nbsp;</span>Add column to DataFrame</a></span></li><li><span><a href="#Dropping-from-DataFrame" data-toc-modified-id="Dropping-from-DataFrame-3.2.8"><span class="toc-item-num">3.2.8&nbsp;&nbsp;</span>Dropping from DataFrame</a></span></li><li><span><a href="#Altering-DataFrame-values" data-toc-modified-id="Altering-DataFrame-values-3.2.9"><span class="toc-item-num">3.2.9&nbsp;&nbsp;</span>Altering DataFrame values</a></span></li><li><span><a href="#Concatenate-two-columns-into-one" data-toc-modified-id="Concatenate-two-columns-into-one-3.2.10"><span class="toc-item-num">3.2.10&nbsp;&nbsp;</span>Concatenate two columns into one</a></span></li><li><span><a href="#Calculating-new-column-conditionally" data-toc-modified-id="Calculating-new-column-conditionally-3.2.11"><span class="toc-item-num">3.2.11&nbsp;&nbsp;</span>Calculating new column conditionally</a></span></li></ul></li><li><span><a href="#Slicing-data-frame" data-toc-modified-id="Slicing-data-frame-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Slicing data frame</a></span><ul class="toc-item"><li><span><a href="#Basic-sclicing" data-toc-modified-id="Basic-sclicing-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>Basic sclicing</a></span></li><li><span><a href="#Conditional-slicing/indexing-(boolean-masking)" data-toc-modified-id="Conditional-slicing/indexing-(boolean-masking)-3.3.2"><span class="toc-item-num">3.3.2&nbsp;&nbsp;</span>Conditional slicing/indexing (boolean masking)</a></span></li><li><span><a href="#Slicing-based-on-partial-string-match-in-column" data-toc-modified-id="Slicing-based-on-partial-string-match-in-column-3.3.3"><span class="toc-item-num">3.3.3&nbsp;&nbsp;</span>Slicing based on partial string match in column</a></span></li><li><span><a href="#Select-rows-by-n-largest/smalles-in-one-column" data-toc-modified-id="Select-rows-by-n-largest/smalles-in-one-column-3.3.4"><span class="toc-item-num">3.3.4&nbsp;&nbsp;</span>Select rows by n largest/smalles in one column</a></span></li></ul></li><li><span><a href="#Iterating-over-DataFrame" data-toc-modified-id="Iterating-over-DataFrame-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Iterating over DataFrame</a></span></li><li><span><a href="#About-chain-indexing" data-toc-modified-id="About-chain-indexing-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>About chain indexing</a></span></li><li><span><a href="#Method-chaining" data-toc-modified-id="Method-chaining-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>Method chaining</a></span></li><li><span><a href="#Data-types-and-scales" data-toc-modified-id="Data-types-and-scales-3.7"><span class="toc-item-num">3.7&nbsp;&nbsp;</span>Data types and scales</a></span></li></ul></li><li><span><a href="#Hierarchical-indexing-(MultiIndex)" data-toc-modified-id="Hierarchical-indexing-(MultiIndex)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Hierarchical indexing (MultiIndex)</a></span><ul class="toc-item"><li><span><a href="#Initialize-DataFrame-with-hierarchical-indexing" data-toc-modified-id="Initialize-DataFrame-with-hierarchical-indexing-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Initialize DataFrame with hierarchical indexing</a></span></li><li><span><a href="#Modifying-MultiIndex-DataFrame" data-toc-modified-id="Modifying-MultiIndex-DataFrame-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Modifying MultiIndex DataFrame</a></span><ul class="toc-item"><li><span><a href="#Add-second-index-to-existing-DataFrame" data-toc-modified-id="Add-second-index-to-existing-DataFrame-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>Add second index to existing DataFrame</a></span></li><li><span><a href="#Add-new-column-to-existing-MultiIndex-DataFrame" data-toc-modified-id="Add-new-column-to-existing-MultiIndex-DataFrame-4.2.2"><span class="toc-item-num">4.2.2&nbsp;&nbsp;</span>Add new column to existing MultiIndex DataFrame</a></span></li><li><span><a href="#Get-different-MultiIndex-levels" data-toc-modified-id="Get-different-MultiIndex-levels-4.2.3"><span class="toc-item-num">4.2.3&nbsp;&nbsp;</span>Get different MultiIndex levels</a></span></li><li><span><a href="#Rename-MultiIndex-columns" data-toc-modified-id="Rename-MultiIndex-columns-4.2.4"><span class="toc-item-num">4.2.4&nbsp;&nbsp;</span>Rename MultiIndex columns</a></span></li></ul></li><li><span><a href="#Quering-MultiIndex-DataFrames" data-toc-modified-id="Quering-MultiIndex-DataFrames-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Quering MultiIndex DataFrames</a></span></li></ul></li><li><span><a href="#Data-wrangling-tricks" data-toc-modified-id="Data-wrangling-tricks-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Data wrangling tricks</a></span><ul class="toc-item"><li><span><a href="#Mappings-for-column-values" data-toc-modified-id="Mappings-for-column-values-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Mappings for column values</a></span><ul class="toc-item"><li><span><a href="#One-to-one-mappings" data-toc-modified-id="One-to-one-mappings-5.1.1"><span class="toc-item-num">5.1.1&nbsp;&nbsp;</span>One-to-one mappings</a></span><ul class="toc-item"><li><span><a href="#Map-from-dict" data-toc-modified-id="Map-from-dict-5.1.1.1"><span class="toc-item-num">5.1.1.1&nbsp;&nbsp;</span>Map from dict</a></span></li><li><span><a href="#Assign-unique-identifier-to-column-values" data-toc-modified-id="Assign-unique-identifier-to-column-values-5.1.1.2"><span class="toc-item-num">5.1.1.2&nbsp;&nbsp;</span>Assign unique identifier to column values</a></span></li><li><span><a href="#Map-column-string-values-to-lowercase" data-toc-modified-id="Map-column-string-values-to-lowercase-5.1.1.3"><span class="toc-item-num">5.1.1.3&nbsp;&nbsp;</span>Map column string values to lowercase</a></span></li><li><span><a href="#Using-regex-with-map" data-toc-modified-id="Using-regex-with-map-5.1.1.4"><span class="toc-item-num">5.1.1.4&nbsp;&nbsp;</span>Using regex with map</a></span></li></ul></li><li><span><a href="#Map-values-to-ranges/groups" data-toc-modified-id="Map-values-to-ranges/groups-5.1.2"><span class="toc-item-num">5.1.2&nbsp;&nbsp;</span>Map values to ranges/groups</a></span></li><li><span><a href="#Categorical-variable-to-dummy-columns" data-toc-modified-id="Categorical-variable-to-dummy-columns-5.1.3"><span class="toc-item-num">5.1.3&nbsp;&nbsp;</span>Categorical variable to dummy columns</a></span></li></ul></li><li><span><a href="#Change-between-long/wide-format" data-toc-modified-id="Change-between-long/wide-format-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Change between long/wide format</a></span><ul class="toc-item"><li><span><a href="#Wide-to-long" data-toc-modified-id="Wide-to-long-5.2.1"><span class="toc-item-num">5.2.1&nbsp;&nbsp;</span>Wide to long</a></span></li><li><span><a href="#Long-to-wide" data-toc-modified-id="Long-to-wide-5.2.2"><span class="toc-item-num">5.2.2&nbsp;&nbsp;</span>Long to wide</a></span></li></ul></li><li><span><a href="#Database-like-joins" data-toc-modified-id="Database-like-joins-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Database-like joins</a></span></li><li><span><a href="#Aggregating-infromation-in-DataFrame" data-toc-modified-id="Aggregating-infromation-in-DataFrame-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Aggregating infromation in DataFrame</a></span><ul class="toc-item"><li><span><a href="#Simple-means,-medians-etc." data-toc-modified-id="Simple-means,-medians-etc.-5.4.1"><span class="toc-item-num">5.4.1&nbsp;&nbsp;</span>Simple means, medians etc.</a></span></li><li><span><a href="#Apply-means,-medians-etc.-to-subportion-of-data-frame" data-toc-modified-id="Apply-means,-medians-etc.-to-subportion-of-data-frame-5.4.2"><span class="toc-item-num">5.4.2&nbsp;&nbsp;</span>Apply means, medians etc. to subportion of data frame</a></span></li><li><span><a href="#'Group-by'-in-various-forms" data-toc-modified-id="'Group-by'-in-various-forms-5.4.3"><span class="toc-item-num">5.4.3&nbsp;&nbsp;</span>'Group by' in various forms</a></span><ul class="toc-item"><li><span><a href="#Table-of-counts/means-for-categorical-variable" data-toc-modified-id="Table-of-counts/means-for-categorical-variable-5.4.3.1"><span class="toc-item-num">5.4.3.1&nbsp;&nbsp;</span>Table of counts/means for categorical variable</a></span></li><li><span><a href="#Table-of-counts-for-continuous-variable" data-toc-modified-id="Table-of-counts-for-continuous-variable-5.4.3.2"><span class="toc-item-num">5.4.3.2&nbsp;&nbsp;</span>Table of counts for continuous variable</a></span></li><li><span><a href="#Weighted-average-with-group-by" data-toc-modified-id="Weighted-average-with-group-by-5.4.3.3"><span class="toc-item-num">5.4.3.3&nbsp;&nbsp;</span>Weighted average with group by</a></span></li><li><span><a href="#SQL-like-group-by---sum-combo" data-toc-modified-id="SQL-like-group-by---sum-combo-5.4.3.4"><span class="toc-item-num">5.4.3.4&nbsp;&nbsp;</span>SQL-like group by - sum combo</a></span></li></ul></li><li><span><a href="#Agg-method" data-toc-modified-id="Agg-method-5.4.4"><span class="toc-item-num">5.4.4&nbsp;&nbsp;</span>Agg method</a></span><ul class="toc-item"><li><span><a href="#Corollary-to-agg-method:-flatten-multi-indexed-frame-for-later-use" data-toc-modified-id="Corollary-to-agg-method:-flatten-multi-indexed-frame-for-later-use-5.4.4.1"><span class="toc-item-num">5.4.4.1&nbsp;&nbsp;</span>Corollary to agg method: flatten multi-indexed frame for later use</a></span></li></ul></li><li><span><a href="#Pivot-table" data-toc-modified-id="Pivot-table-5.4.5"><span class="toc-item-num">5.4.5&nbsp;&nbsp;</span>Pivot table</a></span></li><li><span><a href="#List-observations-with-count->-n" data-toc-modified-id="List-observations-with-count->-n-5.4.6"><span class="toc-item-num">5.4.6&nbsp;&nbsp;</span>List observations with count &gt; n</a></span></li></ul></li></ul></li><li><span><a href="#Data-cleaning" data-toc-modified-id="Data-cleaning-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Data cleaning</a></span><ul class="toc-item"><li><span><a href="#Find-and-replace-string-in-data-frame-column" data-toc-modified-id="Find-and-replace-string-in-data-frame-column-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Find and replace string in data frame column</a></span></li><li><span><a href="#Replace-garbage-values-with-NaN" data-toc-modified-id="Replace-garbage-values-with-NaN-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Replace garbage values with NaN</a></span><ul class="toc-item"><li><span><a href="#Replace-non-numerical-values-in-numerical-colun-with-NaN" data-toc-modified-id="Replace-non-numerical-values-in-numerical-colun-with-NaN-6.2.1"><span class="toc-item-num">6.2.1&nbsp;&nbsp;</span>Replace non-numerical values in numerical colun with NaN</a></span></li><li><span><a href="#Replace-non-numerical-values-in-string-colun-with-NaN" data-toc-modified-id="Replace-non-numerical-values-in-string-colun-with-NaN-6.2.2"><span class="toc-item-num">6.2.2&nbsp;&nbsp;</span>Replace non-numerical values in string colun with NaN</a></span></li></ul></li><li><span><a href="#Find-NaN-values-from-DataFrame" data-toc-modified-id="Find-NaN-values-from-DataFrame-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Find NaN values from DataFrame</a></span></li><li><span><a href="#Clean-NaN-values" data-toc-modified-id="Clean-NaN-values-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Clean NaN values</a></span></li><li><span><a href="#Drop-rows/columns-with-NaN-values" data-toc-modified-id="Drop-rows/columns-with-NaN-values-6.5"><span class="toc-item-num">6.5&nbsp;&nbsp;</span>Drop rows/columns with NaN values</a></span></li><li><span><a href="#Drop-duplicates" data-toc-modified-id="Drop-duplicates-6.6"><span class="toc-item-num">6.6&nbsp;&nbsp;</span>Drop duplicates</a></span></li><li><span><a href="#Find-nonnumeric-values-from-string-column" data-toc-modified-id="Find-nonnumeric-values-from-string-column-6.7"><span class="toc-item-num">6.7&nbsp;&nbsp;</span>Find nonnumeric values from string column</a></span></li></ul></li><li><span><a href="#Working-with-time-and-dates" data-toc-modified-id="Working-with-time-and-dates-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Working with time and dates</a></span><ul class="toc-item"><li><span><a href="#Time-and-date-data-types" data-toc-modified-id="Time-and-date-data-types-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Time and date data types</a></span><ul class="toc-item"><li><span><a href="#Working-with-Period-and-PeriodIndex" data-toc-modified-id="Working-with-Period-and-PeriodIndex-7.1.1"><span class="toc-item-num">7.1.1&nbsp;&nbsp;</span>Working with Period and PeriodIndex</a></span></li></ul></li><li><span><a href="#Date-columns-in-data-frame" data-toc-modified-id="Date-columns-in-data-frame-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Date columns in data frame</a></span><ul class="toc-item"><li><span><a href="#Convert-string-column-to-datetime" data-toc-modified-id="Convert-string-column-to-datetime-7.2.1"><span class="toc-item-num">7.2.1&nbsp;&nbsp;</span>Convert string column to datetime</a></span></li><li><span><a href="#Convert-datetime-to--formatted-string" data-toc-modified-id="Convert-datetime-to--formatted-string-7.2.2"><span class="toc-item-num">7.2.2&nbsp;&nbsp;</span>Convert datetime to  formatted string</a></span></li><li><span><a href="#Harmonize-dates-in-data-frame-column" data-toc-modified-id="Harmonize-dates-in-data-frame-column-7.2.3"><span class="toc-item-num">7.2.3&nbsp;&nbsp;</span>Harmonize dates in data frame column</a></span></li></ul></li><li><span><a href="#Time-series-tricks" data-toc-modified-id="Time-series-tricks-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Time series tricks</a></span><ul class="toc-item"><li><span><a href="#Basic-operations" data-toc-modified-id="Basic-operations-7.3.1"><span class="toc-item-num">7.3.1&nbsp;&nbsp;</span>Basic operations</a></span></li><li><span><a href="#Get-mean-value-per-day-over-multiple-years" data-toc-modified-id="Get-mean-value-per-day-over-multiple-years-7.3.2"><span class="toc-item-num">7.3.2&nbsp;&nbsp;</span>Get mean value per day over multiple years</a></span></li><li><span><a href="#Lead-and-lag" data-toc-modified-id="Lead-and-lag-7.3.3"><span class="toc-item-num">7.3.3&nbsp;&nbsp;</span>Lead and lag</a></span></li></ul></li></ul></li><li><span><a href="#Data-frame-styling" data-toc-modified-id="Data-frame-styling-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Data frame styling</a></span><ul class="toc-item"><li><span><a href="#Global-stylings" data-toc-modified-id="Global-stylings-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Global stylings</a></span><ul class="toc-item"><li><span><a href="#Setting-number-display-format" data-toc-modified-id="Setting-number-display-format-8.1.1"><span class="toc-item-num">8.1.1&nbsp;&nbsp;</span>Setting number display format</a></span></li></ul></li><li><span><a href="#Style-frame-whrn-displaying" data-toc-modified-id="Style-frame-when-displaying-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>Style frame when displaying</a></span></li></ul></li><li><span><a href="#Uncategorized" data-toc-modified-id="Uncategorized-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Uncategorized</a></span><ul class="toc-item"><li><span><a href="#Re-index-columns-to-start-e.g-from-100" data-toc-modified-id="Re-index-columns-to-start-e.g-from-100-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>Re-index columns to start e.g from 100</a></span></li><li><span><a href="#Usual-data-describing-stuff" data-toc-modified-id="Usual-data-describing-stuff-9.2"><span class="toc-item-num">9.2&nbsp;&nbsp;</span>Usual data describing stuff</a></span></li><li><span><a href="#Check-monotonicity-of-column" data-toc-modified-id="Check-monotonicity-of-column-9.3"><span class="toc-item-num">9.3&nbsp;&nbsp;</span>Check monotonicity of column</a></span></li></ul></li></ul></div>

In [1]:
############################################################
# Run this cell!
# Dependencies, auxiliary functions etc.
############################################################
import pandas as pd
import numpy as np
from IPython.display import display_html
from IPython.display import display
import datetime
import random
print("pandas vesion {}".format(pd.__version__))

def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    html_str = html_str.replace('table','table style="display:inline; border:0px"')          
    display_html(html_str,raw=True)

pandas vesion 0.25.0


# Input/output data

## Excel files

### Import Excel file

In [ ]:
# Example applies to Windows paths
#import os
#path = r'C:\myfolder'
#file = 'myfile.xlsx'
#df = pd.read_excel(os.path.join(path,file)
#                   ,sheetname = 'Sheet1'
#                   ,skiprows = 16
#                   ,parse_cols = ("A,C:F")
#                   ,header = None
#                   ,index_col = 0)

### Import csv

In [ ]:
#file_name = "some_file_name.csv"
#dtype = {
#    'some_text_field':object,
#    'some_numeric_field':np.float64,
#}
#dateparse = lambda x: pd.datetime.strptime(x,'%d.%m.%Y')
#df = pd.read_csv(file_name, sep=';',
#                 encoding="ISO-8859-1",
#                 dtype=dtype,
#                 decimal=",",
#                 parse_dates=['soome_data_field'],
#                 date_parser=dateparse)

### Export Excel file
Styling: http://pbpython.com/improve-pandas-excel-output.html

#### Creates new/rewrites whole old book

In [ ]:
# Example applies to Windows paths
#import os
#path = r'C:\myfolder'
#file = 'myfile.xlsx'
#writer = pd.ExcelWriter(os.path.join(path,file))
#df.to_excel(writer, 'Sheet1', header = True)
#writer.save()
#writer.close()

#### Only rewrites range in chosen sheet, not othe sheets
Other sheets and data on selected sheet outside range are preserved.<br>

In [ ]:
#from openpyxl import load_workbook
#import os
#path = r'C:\myfolder'
#file = 'myfile.xlsx'
#book = load_workbook(os.path.join(path,file))
#writer = pd.ExcelWriter(os.path.join(path,file), engine = 'openpyxl')
#writer.book = book
#writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
#dfto_excel(writer, "sheetname", header = True, index=False, startrow = 0, startcol = 0)
#writer.save()
#writer.close()

#### Rewrites chosen sheet but not other sheets

In [ ]:
#from openpyxl import load_workbook
#import os
#path = r'C:\myfolder'
#file = 'myfile.xlsx'
#book = load_workbook(os.path.join(path,file))
#if "sheetname" in book.sheetnames:
#    del book["sheetname"]
#    book.create_sheet("sheetname")
#writer = pd.ExcelWriter(os.path.join(path,file), engine = 'openpyxl')
#writer.book = book
#writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
#df_w.to_excel(writer, "sheetname", header = True, index=False, startrow = 0, startcol = 0)
#writer.save()
#writer.close()

## SQL

### Import SQL table

In [ ]:
#import pyodbc
#import pandas as pd
#
#cnxn = pyodbc.connect('Driver={SQL Server};'
#                      'Server=server_name;'
#                      'Database=dbo_name;'
#                      'Trusted_Connection=yes;')
#cursor = cnxn.cursor()
#sql = "SELECT TOP 1000 * FROM dbo.table"
#df = pd.read_sql_query(sql, cnxn)
#cnxn.close()

### Export data frame to SQL table (MS SQL Server)

In [ ]:
from sqlalchemy import create_engine
import urllib
import pyodbc
quoted = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER={server_name};DATABASE={db_name}")
engine = create_engine("mssql+pyodbc:///?odbc_connect={}".format(quoted))
df.to_sql(name=table_name, schema=schema_name, index=False, con=engine)
engine.dispose()

## Output Latex table

In [22]:
df = pd.DataFrame(np.random.rand(4,4), columns = ['ff', 'ee', 'tt', 'uu'])

# buf can be path, in which case a file is written
df_latex = df.to_latex(buf=None)

# From pandas 1.0.0 onward one can add caption via keyword
# In prior versions one needs to add it manually. After this
# file can be written as per usual
table_name = "tab:mytab"
caption = "This is caption."
df_latex_wcap = df_latex + "".join([
    r"\caption{\label",
    f"{{{table_name}}}",
    f"{caption}",
    r"}",
])

# For manual writing
# with open (path, "w") as f:
#    f.write(df_latex_wcap)

print(df_latex)
print(df_latex_wcap)

\begin{tabular}{lrrrr}
\toprule
{} &        ff &        ee &        tt &        uu \\
\midrule
0 &  0.166407 &  0.846739 &  0.346225 &  0.120631 \\
1 &  0.321834 &  0.357054 &  0.749392 &  0.267605 \\
2 &  0.004588 &  0.395332 &  0.504552 &  0.556089 \\
3 &  0.542579 &  0.504179 &  0.643147 &  0.452426 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrr}
\toprule
{} &        ff &        ee &        tt &        uu \\
\midrule
0 &  0.166407 &  0.846739 &  0.346225 &  0.120631 \\
1 &  0.321834 &  0.357054 &  0.749392 &  0.267605 \\
2 &  0.004588 &  0.395332 &  0.504552 &  0.556089 \\
3 &  0.542579 &  0.504179 &  0.643147 &  0.452426 \\
\bottomrule
\end{tabular}
\caption{\label{tab:mytab}This is caption.}


# Basic DataFrame operations

## DataFrame initialization

In [2]:
# Initialize DataFrame with dummy data and custom indices
df = pd.DataFrame(np.random.rand(4,4), columns = ['ff', 'ee', 'tt', 'uu'])
display(df)

,ff,ee,tt,uu
0,0.493093,0.410482,0.009453,0.390134
1,0.665349,0.794965,0.440991,0.672332
2,0.144569,0.533048,0.366414,0.414370
3,0.264209,0.316266,0.244053,0.163648


In [3]:
# Initialize DataFrame with custom data, with inferred data types
data = {'ID': [1, 2, 3], 'Names': ['Kate', 'John', 'Max'], 'Age': [50, 25, 41], 'Values': [3, 4, 8]}
df = pd.DataFrame(data=data)

# Set one of the columns as index
df2 = df.set_index('ID')
 # Remove index name
df2.index.name = None

# Re-setting index
df3 = df2.copy()
df3.reset_index(inplace = True, drop = True) # drop = True; denies addition of old index to columns

# Force dtypes of columns upon initialization of the frame
data = {'Age': pd.Series([50, 25, 41],dtype=np.int64), 'Values': pd.Series([3.5, 4.4, 8.0],dtype=np.float64)}
df4 = pd.DataFrame(data=data)

display_side_by_side(df,df2,df3,df4)

ID 
 Names 
 Age 
 Values 
 
 
 
 
 0 
 1 
 Kate 
 50 
 3 
 
 
 1 
 2 
 John 
 25 
 4 
 
 
 2 
 3 
 Max 
 41 
 8 
 
 
 
 
 
 
 Names 
 Age 
 Values 
 
 
 
 
 1 
 Kate 
 50 
 3 
 
 
 2 
 John 
 25 
 4 
 
 
 3 
 Max 
 41 
 8 
 
 
 
 
 
 
 Names 
 Age 
 Values 
 
 
 
 
 0 
 Kate 
 50 
 3 
 
 
 1 
 John 
 25 
 4 
 
 
 2 
 Max 
 41 
 8 
 
 
 
 
 
 
 Age 
 Values 
 
 
 
 
 0 
 50 
 3.5 
 
 
 1 
 25 
 4.4 
 
 
 2 
 41 
 8.0

## Modifying DataFrame

### Renaming columns

In [4]:
data = {'ID': [1, 2, 3],'Names': ['Kate', 'John', 'Max'],'Age': [50, 25, 41], 'Values': [3, 4, 8]}
df = pd.DataFrame(data=data)
df2 = df.copy()
df.rename(columns = {"Names": "Name", "Age": "Alter"}, inplace = True)
display_side_by_side(df2,df)

,ID,Names,Age,Values
0,1,Kate,50,3
1,2,John,25,4
2,3,Max,41,8
,ID,Name,Alter,Values
0,1,Kate,50,3
1,2,John,25,4
2,3,Max,41,8


### Sort data frame by colum values

In [5]:
data = {'ID': [1, 2, 3],'Names': ['Kate', 'John', 'Max'],'Age': [50, 25, 25], 'Values': [3, 4, 8]}
df = pd.DataFrame(data=data)

df2 = df.sort_values(['Age','Names'], ascending  = [True, False]).copy()

display_side_by_side(df,df2)

,ID,Names,Age,Values
0,1,Kate,50,3
1,2,John,25,4
2,3,Max,25,8
,ID,Names,Age,Values
2,3,Max,25,8
1,2,John,25,4
0,1,Kate,50,3


### Sort data frame by colums, custom order

In [6]:
data = {'ID': [1, 2, 3],'Names': ['Kate', 'John', 'Max'],'Age': [50, 25, 41], 'Values': [3, 4, 8]}
df = pd.DataFrame(data=data)
df2 = df.copy()
df2.index = pd.Categorical(df2.index, [2,0,1])
df2 = df2.sort_index(axis = 0)
display_side_by_side(df, df2)

,ID,Names,Age,Values
0,1,Kate,50,3
1,2,John,25,4
2,3,Max,41,8
,ID,Names,Age,Values
2,3,Max,41,8
0,1,Kate,50,3
1,2,John,25,4


### Change DataFrame column order

In [7]:
data = {'ID': [1, 2,], 'Names': ['John', 'Max'], 'Vals1': [50, 41], 'Vals2': [4.0, 8.0], 'Vals3': ['4.4', '3.1']}
df = pd.DataFrame(data = data)
df2 = df.iloc[:, ::-1].copy()
df3 = df[['Vals2', 'Names', 'Vals3', 'Vals1']].copy()
display_side_by_side(df,df2,df3)

ID 
 Names 
 Vals1 
 Vals2 
 Vals3 
 
 
 
 
 0 
 1 
 John 
 50 
 4.0 
 4.4 
 
 
 1 
 2 
 Max 
 41 
 8.0 
 3.1 
 
 
 
 
 
 
 Vals3 
 Vals2 
 Vals1 
 Names 
 ID 
 
 
 
 
 0 
 4.4 
 4.0 
 50 
 John 
 1 
 
 
 1 
 3.1 
 8.0 
 41 
 Max 
 2 
 
 
 
 
 
 
 Vals2 
 Names 
 Vals3 
 Vals1 
 
 
 
 
 0 
 4.0 
 John 
 4.4 
 50 
 
 
 1 
 8.0 
 Max 
 3.1 
 41

### Changing data types

In [8]:
# A type 'O' just stands for "object" which in Pandas' world is a string (characters). 

data = {'ID': [1, 2, 3],'Names': ['Kate', 'John', 'Max']
        ,'Vals1': [50, 25, 41], 'Vals2': [3.0, 4.0, 8.0], 'Vals3': ['3.0', '4.4', '3.1'], 'Vals4': [3.0, np.nan, 3.1]}
df = pd.DataFrame(data=data)
df2 = df.copy()

# Here we convert to numpy data types instead of native python types (float/int)
# Numpy data types are of fixed size but are more efficient
df2['Vals1'] = df2['Vals1'].astype('float64')
df2['Vals2'] = df2['Vals2'].astype('int64')
df2['Vals3'] = df2['Vals3'].astype('float64') # string to float, see also pd.to_numeric

# convert to strong but preserve NaNs
df2.loc[df2['Vals4'].notnull(), 'Vals4'] = df2['Vals4'].astype(str)

# Check
print("Before Vals4 type was {}".format(df['Vals4'].dtype))
print("After Vals4 type was {}".format(df2['Vals4'].dtype))

print("This should be str: {}".format(type(df2.loc[0,'Vals4'])))
print("This should be float: {}".format(type(df2.loc[1,'Vals4'])))

display_side_by_side(df,df2)

Before Vals4 type was float64
After Vals4 type was object
This should be str: <class 'str'>
This should be float: <class 'float'>


,ID,Names,Vals1,Vals2,Vals3,Vals4
0,1,Kate,50,3.0,3.0,3.0
1,2,John,25,4.0,4.4,NaN
2,3,Max,41,8.0,3.1,3.1
,ID,Names,Vals1,Vals2,Vals3,Vals4
0,1,Kate,50.0,3,3.0,3.0
1,2,John,25.0,4,4.4,NaN
2,3,Max,41.0,8,3.1,3.1


### Add row to DataFrame

In [9]:
data = {'ID': [1, 2,], 'Names': ['John', 'Max'],'Vals1': [50, 41], 'Vals2': [4.0, 8.0], 'Vals3': ['4.4', '3.1']}
df = pd.DataFrame(data = data)

df2 = pd.DataFrame({'ID': [4], 'Names': ['Janet'], 'Vals1': [50], 'Vals2': [12.0], 'Vals3': ['2.9']})
df3 = df.append(df2, ignore_index = True)

display_side_by_side(df,df3)

,ID,Names,Vals1,Vals2,Vals3
0,1,John,50,4.0,4.4
1,2,Max,41,8.0,3.1
,ID,Names,Vals1,Vals2,Vals3
0,1,John,50,4.0,4.4
1,2,Max,41,8.0,3.1
2,4,Janet,50,12.0,2.9


### Add column to DataFrame

In [10]:
df = pd.DataFrame(data = {'ID': [1, 2, 3], 'Names': ['John', 'Max', 'Jane'],'Vals1': [50, 41, 32]})
df2 = pd.DataFrame(data = {'Vals2': [0.8, 0.6, 0.4]})
df3 = pd.concat([df, df2], axis = 1)
display_side_by_side(df, df2, df3)

ID 
 Names 
 Vals1 
 
 
 
 
 0 
 1 
 John 
 50 
 
 
 1 
 2 
 Max 
 41 
 
 
 2 
 3 
 Jane 
 32 
 
 
 
 
 
 
 Vals2 
 
 
 
 
 0 
 0.8 
 
 
 1 
 0.6 
 
 
 2 
 0.4 
 
 
 
 
 
 
 ID 
 Names 
 Vals1 
 Vals2 
 
 
 
 
 0 
 1 
 John 
 50 
 0.8 
 
 
 1 
 2 
 Max 
 41 
 0.6 
 
 
 2 
 3 
 Jane 
 32 
 0.4

### Dropping from DataFrame

In [11]:
data = {'ID': [1, 2, 3, 4, 5, 6]
        ,'Names': ['Kate', 'John', 'Max', 'Mary', 'Pete', 'Miriam']
        ,'Age': [50, 25, 41, 99, 54, 23]
        ,'Values': [3, 4, 8, 4, 3, 4]
        ,'col': [None]}
df_orig = pd.DataFrame(data=data, index = ['a','b','c','d', 'e', 'f'])
df = df_orig.copy()

# Drop columns by column label
df.drop(['Age', 'Names'], axis = 1, inplace = True)

# Drop column by fancy indexing
df.drop(df.columns[0], axis = 1, inplace = True)

# Drop column by using del; is directly inplace!
del df['col']

# Drop rows by index label
df.drop(['b'], axis = 0 , inplace = True)

# Drop rows by index number
df.drop(df.index[[0]], axis = 0, inplace = True)

# Drop rows where value in column does not fulfill condition
df = df[df['Values'] <= 5]

# Drop first n = 1 rows
df.drop(df.head(1).index,inplace=True)

# Drop last n = 1 rows
df.drop(df.tail(1).index,inplace=True) # drop last n rows

display_side_by_side(df_orig,df)

ID 
 Names 
 Age 
 Values 
 col 
 
 
 
 
 a 
 1 
 Kate 
 50 
 3 
 None 
 
 
 b 
 2 
 John 
 25 
 4 
 None 
 
 
 c 
 3 
 Max 
 41 
 8 
 None 
 
 
 d 
 4 
 Mary 
 99 
 4 
 None 
 
 
 e 
 5 
 Pete 
 54 
 3 
 None 
 
 
 f 
 6 
 Miriam 
 23 
 4 
 None 
 
 
 
 
 
 
 Values 
 
 
 
 
 e 
 3

In [12]:
# Drop columns except those we want to keep
data = {'ID': [1, 2, 3, 4], 'Names': ['Kate', 'John', 'Max', 'Mary'], 'Age': [50, 25, 41,99], 'Values': [3, 4, 8,4], 'col': [None]}
df = pd.DataFrame(data = data, index = ['a','b','c','d'])
columns_to_keep = ['Names', 'Values']
df2 = df[columns_to_keep].copy()
display_side_by_side(df, df2)

ID 
 Names 
 Age 
 Values 
 col 
 
 
 
 
 a 
 1 
 Kate 
 50 
 3 
 None 
 
 
 b 
 2 
 John 
 25 
 4 
 None 
 
 
 c 
 3 
 Max 
 41 
 8 
 None 
 
 
 d 
 4 
 Mary 
 99 
 4 
 None 
 
 
 
 
 
 
 Names 
 Values 
 
 
 
 
 a 
 Kate 
 3 
 
 
 b 
 John 
 4 
 
 
 c 
 Max 
 8 
 
 
 d 
 Mary 
 4

### Altering DataFrame values

In [13]:
df = pd.DataFrame(np.arange(12).reshape(6,2), columns = ['ff', 'er'])
df2 = df.copy()

# Apply change to whole column
df2['ff'] *= 0.8

# Conditionally modify certain cells
df2.loc[df2['er'] < 7,'er'] -= 2

# using np.where, create altered column
df2['altered_ff'] = np.where(df2['ff'] < 7, 1, 0)

display_side_by_side(df,df2)

ff 
 er 
 
 
 
 
 0 
 0 
 1 
 
 
 1 
 2 
 3 
 
 
 2 
 4 
 5 
 
 
 3 
 6 
 7 
 
 
 4 
 8 
 9 
 
 
 5 
 10 
 11 
 
 
 
 
 
 
 ff 
 er 
 altered_ff 
 
 
 
 
 0 
 0.0 
 -1 
 1 
 
 
 1 
 1.6 
 1 
 1 
 
 
 2 
 3.2 
 3 
 1 
 
 
 3 
 4.8 
 7 
 1 
 
 
 4 
 6.4 
 9 
 1 
 
 
 5 
 8.0 
 11 
 0

### Concatenate two columns into one

In [14]:
data = {'ID': [1, 2, 3],'Names': ['Kate', 'John', 'Max'], 'Occupation': ['CEO', 'Doctor', 'Driver']
        ,'Vals1': [50, 25, 41], 'Vals2': [3.0, 4.0, 8.0]}
df = pd.DataFrame(data=data)
df2 = df.copy()

df2['two_string_cols'] = df2['Names'] + '_' + df2['Occupation']
df2['str_and_numeric'] = df2['Names'] + '_' + df2['Vals2'].map(str)
df2['format_numerics'] = df2['Names'] + '_' + df2['Vals2'].map("{:.0f}".format)

display_side_by_side(df, df2)

ID 
 Names 
 Occupation 
 Vals1 
 Vals2 
 
 
 
 
 0 
 1 
 Kate 
 CEO 
 50 
 3.0 
 
 
 1 
 2 
 John 
 Doctor 
 25 
 4.0 
 
 
 2 
 3 
 Max 
 Driver 
 41 
 8.0 
 
 
 
 
 
 
 ID 
 Names 
 Occupation 
 Vals1 
 Vals2 
 two_string_cols 
 str_and_numeric 
 format_numerics 
 
 
 
 
 0 
 1 
 Kate 
 CEO 
 50 
 3.0 
 Kate_CEO 
 Kate_3.0 
 Kate_3 
 
 
 1 
 2 
 John 
 Doctor 
 25 
 4.0 
 John_Doctor 
 John_4.0 
 John_4 
 
 
 2 
 3 
 Max 
 Driver 
 41 
 8.0 
 Max_Driver 
 Max_8.0 
 Max_8

### Calculating new column conditionally

In [15]:
data = {'ID': [1, 2, 3],'Names': ['Kate', 'John', 'Max'],'val1': [50, 25, 41], 'val2': [3.0, 0, 8.0]}
df = pd.DataFrame(data=data)
df2 = df.copy()
df3 = df.copy()

df2['val1/val2']  = df2['val1']/df2['val2']
df3['val1/val2']  = np.where(df2['val2'] != 0, df2['val1']/df2['val2'], np.nan)
df2['val1']/df2['val2']

display_side_by_side(df, df2, df3)

ID 
 Names 
 val1 
 val2 
 
 
 
 
 0 
 1 
 Kate 
 50 
 3.0 
 
 
 1 
 2 
 John 
 25 
 0.0 
 
 
 2 
 3 
 Max 
 41 
 8.0 
 
 
 
 
 
 
 ID 
 Names 
 val1 
 val2 
 val1/val2 
 
 
 
 
 0 
 1 
 Kate 
 50 
 3.0 
 16.666667 
 
 
 1 
 2 
 John 
 25 
 0.0 
 inf 
 
 
 2 
 3 
 Max 
 41 
 8.0 
 5.125000 
 
 
 
 
 
 
 ID 
 Names 
 val1 
 val2 
 val1/val2 
 
 
 
 
 0 
 1 
 Kate 
 50 
 3.0 
 16.666667 
 
 
 1 
 2 
 John 
 25 
 0.0 
 NaN 
 
 
 2 
 3 
 Max 
 41 
 8.0 
 5.125000

## Slicing data frame

### Basic sclicing
Using data[2] or data['label'] indexes the series itself, skipping the iloc/loc attributes. This is syntatic sugar, but it is safer to use iloc/loc attributes explicitly in order avoid confusion querying between index label and index position.

In [16]:
df = pd.DataFrame(np.random.rand(4,4), columns = ['ff', 'ee', 'tt', 'uuu'], index = ['first','second','third','fourth'])

# Selecting columns
# Using single square brackets selects a Series from DataFrame. 
# Using multiple square brackets selects a slice of data frame (possibly with muliple columns) 
df['ff'] # returns Series
df[['ff']] # returns DataFrame with one column
df[['ff', 'ee']] # returns DataFrame with two columns
df.loc[:,['ee', 'tt']]
df.iloc[:,0] # by position in frame
df.iloc[:,0:3] # first 3 columns

# Selecting rows
df.loc['first'] # by index
df.loc[~df.index.isin(['first'])] # rows except index 'first'
df.loc[['first', 'fourth']] # by index multiple rows
df[0:3]
df.iloc[0:2] # by position in frame

# Select ith row of column 'ee'
df.iloc[2:4,list(df.columns).index('ee')]

display_side_by_side(df, df.loc[:,['ee','tt']], df.iloc[0:2], df.loc[~df.index.isin(['first'])], df.iloc[:,0:3],
                     pd.DataFrame(df.iloc[2:4,list(df.columns).index('ee')]))

ff 
 ee 
 tt 
 uuu 
 
 
 
 
 first 
 0.326338 
 0.407145 
 0.188352 
 0.153224 
 
 
 second 
 0.486314 
 0.324041 
 0.309229 
 0.053088 
 
 
 third 
 0.201859 
 0.829955 
 0.491291 
 0.261846 
 
 
 fourth 
 0.294076 
 0.805226 
 0.949530 
 0.014044 
 
 
 
 
 
 
 ee 
 tt 
 
 
 
 
 first 
 0.407145 
 0.188352 
 
 
 second 
 0.324041 
 0.309229 
 
 
 third 
 0.829955 
 0.491291 
 
 
 fourth 
 0.805226 
 0.949530 
 
 
 
 
 
 
 ff 
 ee 
 tt 
 uuu 
 
 
 
 
 first 
 0.326338 
 0.407145 
 0.188352 
 0.153224 
 
 
 second 
 0.486314 
 0.324041 
 0.309229 
 0.053088 
 
 
 
 
 
 
 ff 
 ee 
 tt 
 uuu 
 
 
 
 
 second 
 0.486314 
 0.324041 
 0.309229 
 0.053088 
 
 
 third 
 0.201859 
 0.829955 
 0.491291 
 0.261846 
 
 
 fourth 
 0.294076 
 0.805226 
 0.949530 
 0.014044 
 
 
 
 
 
 
 ff 
 ee 
 tt 
 
 
 
 
 first 
 0.326338 
 0.407145 
 0.188352 
 
 
 second 
 0.486314 
 0.324041 
 0.309229 
 
 
 third 
 0.201859 
 0.829955 
 0.491291 
 
 
 fourth 
 0.294076 
 0.805226 
 0.949530 
 
 
 
 
 
 
 ee 
 
 
 
 
 third 
 0.829955 
 
 
 fourth 
 0.805226

### Conditional slicing/indexing (boolean masking)

In [17]:
data = {'ID': [1, 2, 3, 4], 'Names': ['Kate', 'John', 'Max', 'Mary'], 'Age': [50, 25, 41,99], 'Values': [3, 4, 8,4]}
df = pd.DataFrame(data=data, index = ['a','b','c','d'])

# Get row slice fulfilling a condition along one column, return only
# certain columns
df2 = pd.DataFrame(df.loc[df['Values'] < 5,  ['Names', 'Values']].copy())

# Same as above but multiple conditions, return all columns tha match
df3 = df.loc[(df['Values'] < 5 ) & (df['Names'] == 'John'), ].copy()

# Conditional indexing by preserving original dimensions
df4 = df.where(df['Values'] < 5).copy()

display_side_by_side(df,df2,df3,df4)

ID 
 Names 
 Age 
 Values 
 
 
 
 
 a 
 1 
 Kate 
 50 
 3 
 
 
 b 
 2 
 John 
 25 
 4 
 
 
 c 
 3 
 Max 
 41 
 8 
 
 
 d 
 4 
 Mary 
 99 
 4 
 
 
 
 
 
 
 Names 
 Values 
 
 
 
 
 a 
 Kate 
 3 
 
 
 b 
 John 
 4 
 
 
 d 
 Mary 
 4 
 
 
 
 
 
 
 ID 
 Names 
 Age 
 Values 
 
 
 
 
 b 
 2 
 John 
 25 
 4 
 
 
 
 
 
 
 ID 
 Names 
 Age 
 Values 
 
 
 
 
 a 
 1.0 
 Kate 
 50.0 
 3.0 
 
 
 b 
 2.0 
 John 
 25.0 
 4.0 
 
 
 c 
 NaN 
 NaN 
 NaN 
 NaN 
 
 
 d 
 4.0 
 Mary 
 99.0 
 4.0

### Slicing based on partial string match in column

In [18]:
data = {'ID': [1, 2, 3, 4], 'Names': ['Kare', 'John', 'Max', 'Mary'], 'Age': [50, 25, 41,99], 'Values': [3, 4, 8,4]}
df = pd.DataFrame(data=data, index = ['a','b','c','d'])
df2 = df.copy()

df2 = df2[df2['Names'].str.contains("ar")]

display_side_by_side(df, df2)

,ID,Names,Age,Values
a,1,Kare,50,3
b,2,John,25,4
c,3,Max,41,8
d,4,Mary,99,4
,ID,Names,Age,Values
a,1,Kare,50,3
d,4,Mary,99,4


### Select rows by n largest/smalles in one column
If the nth largest/smallest is a duplicate value, will return al of those. The way here is most deninitely not the most elegant solution...

In [19]:
data = {'ID': [1, 2, 3, 4, 5], 'Names': ['Kare', 'John', 'Max', 'Mary', 'Santa'], 'Age': [50, 25, 41,99, 23], 'Values': [3, 4, 8, 4, 2]}
df = pd.DataFrame(data=data)

df2 = df[df['Values'] >= df['Values'].nlargest(2).values[-1]].copy()
df3 = df[df['Values'] <= df['Values'].nsmallest(3).values[-1]].copy()

display_side_by_side(df,df2,df3)

,ID,Names,Age,Values
0,1,Kare,50,3
1,2,John,25,4
2,3,Max,41,8
3,4,Mary,99,4
4,5,Santa,23,2
,ID,Names,Age,Values
1,2,John,25,4
2,3,Max,41,8
3,4,Mary,99,4
,ID,Names,Age,Values


## Iterating over DataFrame

In [20]:
data = {'Names': ['Kate', 'John', 'Max'], 'Age': [50, 25, 41], 'Weight': [62, 76, 98]}
df = pd.DataFrame(data = data)

# "Right" way with df.iterrows 
for index, row in df.iterrows():
    print(index, row['Age'])

print('-'*30)    

# "Normal" integer looping over rows
for counter in range(0,len(df)):
    print(df.iloc[counter,0])   

0 50
1 25
2 41
------------------------------
Kate
John
Max


In [21]:
# Set values during row iteration
data = {'Names': ['Kate', 'John', 'Max'], 'Age': [50, 25, 41], 'Weight': [62, 76, 98]}
df = pd.DataFrame(data = data)
df2 = df.copy()

for index, row in df2.iterrows():
    if row['Names'] == 'John':
        df2.set_value(index, 'Weight', 100)
display_side_by_side(df,df2)    

,Names,Age,Weight
0,Kate,50,62
1,John,25,76
2,Max,41,98
,Names,Age,Weight
0,Kate,50,62
1,John,25,100
2,Max,41,98


## About chain indexing

In [22]:
# A rule of thub is that any time you see back-to-back square brackets,
# ][, you're asking for trouble. Replace that with a .loc[] or .iloc[]
# and you'll be set. Let's see an example

df = pd.DataFrame(np.random.rand(4,4), columns = ['ff', 'ee', 'tt', 'uuu'], index = ['first','second','third','fourth'])
display(df)
print(df['ee']['second']) # bad practice
print(df.loc['second', 'ee']) # good practice

,ff,ee,tt,uuu
first,0.546698,0.158420,0.492188,0.293475
second,0.899888,0.838052,0.555978,0.462397
third,0.571611,0.951684,0.330207,0.711627
fourth,0.757804,0.550936,0.948268,0.464751


0.8380521374492956
0.8380521374492956


In [23]:
# Another example
df = pd.DataFrame()
df['x'] = np.arange(10,14)
df['value'] = np.arange(200,204)
display(df)

print(df[ df['x']== 10 ]['value']) # bad practice
print('-'*30)
print(df.loc[df['x'] == 10, 'value']) # good practice

# The bad practice option would't even work in the case
# we want to assign a new value for the returned obseravtions

#df[ df['x']== 10 ]['value'] = 1000 # this throws error!
print('-'*30)
df.loc[df['x'] == 10, 'value'] = 1000
display(df)

,x,value
0,10,200
1,11,201
2,12,202
3,13,203


0    200
Name: value, dtype: int32
------------------------------
0    200
Name: value, dtype: int32
------------------------------


,x,value
0,10,1000
1,11,201
2,12,202
3,13,203


## Method chaining

In [24]:
# 'Pandorable' way is to chain methods operating on DataFrame:
data = {'Names': ['Kate', 'John', 'Max'],'Age': [50, 25, 41], 'Weight': [62, 76, 98]}
df = pd.DataFrame(data=data)

df2 = df.copy()
# Not-so-good practice: two lines of code
df2 = df2.drop(df2[df2['Age'] == 25].index)
df2.rename(columns={'Weight': 'Weight (kg)'}, inplace = True)

df3 = df.copy()
# Better practice - on line of code
df3 = df3.drop(df3[df3['Age'] == 25].index).rename(columns = {'Weight': 'Weight (kg)'}).copy()

display_side_by_side(df,df2,df3)

,Names,Age,Weight
0,Kate,50,62
1,John,25,76
2,Max,41,98
,Names,Age,Weight (kg)
0,Kate,50,62
2,Max,41,98
,Names,Age,Weight (kg)
0,Kate,50,62
2,Max,41,98


## Data types and scales

This section is till incomplete!

In general, there are 4 types of data scales
<ol>
  <li>Ratio scale, e.g. height</li>
  <li>Inteval scale, e.g. temperature</li>
  <li>Ordinal scale, e.g. grading scale</li>
  <li>Nominal scale, e.g. school names</li>
</ol>

In [42]:
from pandas.api.types import CategoricalDtype
df = pd.DataFrame(['E','M','L','M','I','A','B','B','M'], columns = ['Grades'])

cats = ['I','A','B','C','M','E','L']
cat_type = CategoricalDtype(categories=cats, ordered=True)
df['Grades'] = df['Grades'].astype(cat_type)

df2 = df.sort_values(['Grades'], ascending  = False).copy()
df3 = df[df['Grades'] >= 'M'].copy()

display_side_by_side(df,df2,df3)


,Grades
0,E
1,M
2,L
3,M
4,I
5,A
6,B
7,B
8,M
,Grades


# Hierarchical indexing (MultiIndex)

## Initialize DataFrame with hierarchical indexing

In [27]:
df = pd.DataFrame(data = np.arange(12).reshape((4,3))
                  ,index = [['a','a','b','b',], [1,2] * 2]
                  ,columns = [['col1','col1','col2'], ['ff','gg','ff']])
display_side_by_side(df)

col1 
 col2 
 
 
 
 
 ff 
 gg 
 ff 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 
 
 2 
 3 
 4 
 5 
 
 
 b 
 1 
 6 
 7 
 8 
 
 
 2 
 9 
 10 
 11

## Modifying MultiIndex DataFrame

### Add second index to existing DataFrame

In [28]:
df = pd.DataFrame(data = np.arange(12).reshape((4,3)), columns = ['col1','col1','col2'])
df2 = df.copy()
df2.set_index([df2.index, 'col2'], inplace = True)
df2.index.names = ['Index1', 'Index2']
display_side_by_side(df,df2)

,col1,col1,col2
0,0,1,2
1,3,4,5
2,6,7,8
3,9,10,11
,,col1,col1
Index1,Index2,,
0,2,0,1
1,5,3,4
2,8,6,7
3,11,9,10


### Add new column to existing MultiIndex DataFrame

In [29]:
df = pd.DataFrame(data = np.arange(16).reshape((4,4))
                  ,index = [['a','a','b','b',], [1,2] * 2]
                  ,columns = [['col1','col1','col2','col2'], ['ff','gg','ff','gg']])

# Add new column to MultiIndex frame
df2 = df.copy()
df2['col2', 'dd'] = [0, 1, 2, 4]
display_side_by_side(df,df2)

col1 
 col2 
 
 
 
 
 ff 
 gg 
 ff 
 gg 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 3 
 
 
 2 
 4 
 5 
 6 
 7 
 
 
 b 
 1 
 8 
 9 
 10 
 11 
 
 
 2 
 12 
 13 
 14 
 15 
 
 
 
 
 
 
 
 col1 
 col2 
 
 
 
 
 ff 
 gg 
 ff 
 gg 
 dd 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 3 
 0 
 
 
 2 
 4 
 5 
 6 
 7 
 1 
 
 
 b 
 1 
 8 
 9 
 10 
 11 
 2 
 
 
 2 
 12 
 13 
 14 
 15 
 4

### Get different MultiIndex levels

In [30]:
df = pd.DataFrame(data = np.arange(12).reshape((4,3))
                  ,index = [['a','a','b','b',], [1,2] * 2]
                  ,columns = [['col1','col1','col2'], ['ff','gg','ff']])

print(df.columns.get_level_values(0))
print(df.columns.get_level_values(1))
print(df.index.get_level_values(0))
print(df.index.get_level_values(1))
print(df.index.levels[1])
print(df.columns.levels[0])
display(df)

Index(['col1', 'col1', 'col2'], dtype='object')
Index(['ff', 'gg', 'ff'], dtype='object')
Index(['a', 'a', 'b', 'b'], dtype='object')
Int64Index([1, 2, 1, 2], dtype='int64')
Int64Index([1, 2], dtype='int64')
Index(['col1', 'col2'], dtype='object')


col1     col2
      ff  gg   ff
a 1    0   1    2
  2    3   4    5
b 1    6   7    8
  2    9  10   11

### Rename MultiIndex columns

In [31]:
df = pd.DataFrame(data = np.arange(12).reshape((4,3))
                  ,index = [['a','a','b','b',], [1,2] * 2]
                  ,columns = [['col1','col1','col2'], ['ff','gg','ff']])
df.rename(columns={"col1": "sss"}, inplace=True, level = 0)
df

sss     col2
     ff  gg   ff
a 1   0   1    2
  2   3   4    5
b 1   6   7    8
  2   9  10   11

### Re-order specific multi-index level 

In [16]:
df = pd.DataFrame(
    data=np.arange(12).reshape((4,3)),
    index=[['a','a','b','b',], [1,2] * 2],
    columns=[['col1','col1','col2'], ['ff','gg','ff']]
)
display_side_by_side(
    df,
    df.reindex(["b", "a"], level=0, axis=0),
    df.reindex(["gg", "ff"], level=1, axis=1),
)

col1 
 col2 
 
 
 
 
 ff 
 gg 
 ff 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 
 
 2 
 3 
 4 
 5 
 
 
 b 
 1 
 6 
 7 
 8 
 
 
 2 
 9 
 10 
 11 
 
 
 
 
 
 
 
 col1 
 col2 
 
 
 
 
 ff 
 gg 
 ff 
 
 
 
 
 b 
 1 
 6 
 7 
 8 
 
 
 2 
 9 
 10 
 11 
 
 
 a 
 1 
 0 
 1 
 2 
 
 
 2 
 3 
 4 
 5 
 
 
 
 
 
 
 
 col1 
 col2 
 
 
 
 
 gg 
 ff 
 ff 
 
 
 
 
 a 
 1 
 1 
 0 
 2 
 
 
 2 
 4 
 3 
 5 
 
 
 b 
 1 
 7 
 6 
 8 
 
 
 2 
 10 
 9 
 11

## Quering MultiIndex DataFrames

In [32]:
df = pd.DataFrame(data = np.arange(16).reshape((4,4))
                  ,index = [['a','a','b','b',], [1,2] * 2]
                  ,columns = [['col1','col1','col2','col2'], ['ff','gg','ff','gg']])

# Get observations from certain column level
df2 = df.iloc[:, df.columns.get_level_values(1) == 'gg'].copy()
df2b = df.iloc[:, df.columns.get_level_values(1).isin(['gg','ee'])].copy()

# Get observations from certain index level
df3 = df.iloc[df.index.get_level_values(1) == 2,:].copy()

# Get observations from certain column levels (multi-condition)
df5 = df.loc(axis=1)[['col1'],:].copy()
df4 = df.iloc[:, (df.columns.get_level_values(0) == 'col1') & (df.columns.get_level_values(1) == 'gg')].copy()

display_side_by_side(df,df2,df2b,df3,df4,df5)

col1 
 col2 
 
 
 
 
 ff 
 gg 
 ff 
 gg 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 3 
 
 
 2 
 4 
 5 
 6 
 7 
 
 
 b 
 1 
 8 
 9 
 10 
 11 
 
 
 2 
 12 
 13 
 14 
 15 
 
 
 
 
 
 
 
 col1 
 col2 
 
 
 
 
 gg 
 gg 
 
 
 
 
 a 
 1 
 1 
 3 
 
 
 2 
 5 
 7 
 
 
 b 
 1 
 9 
 11 
 
 
 2 
 13 
 15 
 
 
 
 
 
 
 
 col1 
 col2 
 
 
 
 
 gg 
 gg 
 
 
 
 
 a 
 1 
 1 
 3 
 
 
 2 
 5 
 7 
 
 
 b 
 1 
 9 
 11 
 
 
 2 
 13 
 15 
 
 
 
 
 
 
 
 col1 
 col2 
 
 
 
 
 ff 
 gg 
 ff 
 gg 
 
 
 
 
 a 
 2 
 4 
 5 
 6 
 7 
 
 
 b 
 2 
 12 
 13 
 14 
 15 
 
 
 
 
 
 
 
 col1 
 
 
 
 
 gg 
 
 
 
 
 a 
 1 
 1 
 
 
 2 
 5 
 
 
 b 
 1 
 9 
 
 
 2 
 13 
 
 
 
 
 
 
 
 col1 
 
 
 
 
 ff 
 gg 
 
 
 
 
 a 
 1 
 0 
 1 
 
 
 2 
 4 
 5 
 
 
 b 
 1 
 8 
 9 
 
 
 2 
 12 
 13

# Data wrangling tricks

## Mappings for column values

### One-to-one mappings

#### Map from dict

In [33]:
data = {'ID': [1, 2, 3,4],'Names': ['Kate', 'John', 'Max','Kate'],'Age': [50, 25, 41,89], 'Values': [3, 4, 8, 12]}
df = pd.DataFrame(data = data)
df2 = df.copy()

# Mapping from user defined dict
age_mapping = {25:0, 41:1, 50:2, 89:3}
df2['AgeOrder'] = df2['Age'].map(age_mapping)
display_side_by_side(df,df2)

ID 
 Names 
 Age 
 Values 
 
 
 
 
 0 
 1 
 Kate 
 50 
 3 
 
 
 1 
 2 
 John 
 25 
 4 
 
 
 2 
 3 
 Max 
 41 
 8 
 
 
 3 
 4 
 Kate 
 89 
 12 
 
 
 
 
 
 
 ID 
 Names 
 Age 
 Values 
 AgeOrder 
 
 
 
 
 0 
 1 
 Kate 
 50 
 3 
 2 
 
 
 1 
 2 
 John 
 25 
 4 
 0 
 
 
 2 
 3 
 Max 
 41 
 8 
 1 
 
 
 3 
 4 
 Kate 
 89 
 12 
 3

#### Assign unique identifier to column values

In [34]:
data = {'ID': [1, 2, 3,4], 'Names': ['Kate', 'John', 'Max','Kate'], 'Age': [50, 25, 41,89], 'Values': [3, 4, 8, 12]}
df = pd.DataFrame(data = data)
df2 = df.copy()

# Assign unique identifier to values in column (Names -> NameID)
d = {key: value for (key, value) in zip(df['Names'].unique(),range(len(df['Names'].unique())))}
df2['NameID'] = df2['Names'].map(d)
display_side_by_side(df,df2)

ID 
 Names 
 Age 
 Values 
 
 
 
 
 0 
 1 
 Kate 
 50 
 3 
 
 
 1 
 2 
 John 
 25 
 4 
 
 
 2 
 3 
 Max 
 41 
 8 
 
 
 3 
 4 
 Kate 
 89 
 12 
 
 
 
 
 
 
 ID 
 Names 
 Age 
 Values 
 NameID 
 
 
 
 
 0 
 1 
 Kate 
 50 
 3 
 0 
 
 
 1 
 2 
 John 
 25 
 4 
 1 
 
 
 2 
 3 
 Max 
 41 
 8 
 2 
 
 
 3 
 4 
 Kate 
 89 
 12 
 0

#### Map column string values to lowercase

In [35]:
data = {'ID': [1, 2, 3,4], 'Names': ['Kate', 'JoHn', 'MA4','KATE'], 'Values': [3, 4, 8, 12]}
df = pd.DataFrame(data = data)
df2 = df.copy()
df2['Names'] = df2['Names'].map(lambda x: x if type(x)!=str else x.lower())

display_side_by_side(df, df2)

,ID,Names,Values
0,1,Kate,3
1,2,JoHn,4
2,3,MA4,8
3,4,KATE,12
,ID,Names,Values
0,1,kate,3
1,2,john,4
2,3,ma4,8
3,4,kate,12


#### Using regex with map

In [36]:
import re
data = {'ID': [1, 2, 3,4], 'Names': ['Kate', 'John', 'Max','Kate'], 'Values': ['foff', 'ss-d', '8', '-rrgr']}
df = pd.DataFrame(data = data)
df2 = df.copy()

# also includes the delimiter!
regex2 = re.compile("(?:-).*")
df2['Values'] = df2['Values'].replace(to_replace = regex2, value = 'REPLACED')

display_side_by_side(df, df2)

,ID,Names,Values
0,1,Kate,foff
1,2,John,ss-d
2,3,Max,8
3,4,Kate,-rrgr
,ID,Names,Values
0,1,Kate,foff
1,2,John,ssREPLACED
2,3,Max,8
3,4,Kate,REPLACED


### Map values to ranges/groups

In [37]:
data = {'ID': [1, 2, 3,4], 'Names': ['Kate', 'John', 'Max','Kate'], 'Age': [39, 18, 25,75], 'Values': [3, 4, 8, 12]}
df = pd.DataFrame(data = data)
df2 = df.copy()

# Even-sized bins 
df2['ValGroup1'] = pd.cut(df2['Values'] , 4, right = True) # 4 bins, includes rightmost edge
                    
# Relative-sized bins 
df2['ValGroup2'] = pd.qcut(df2['Values'], 4)

# Custom bin sizes and labels
df2['AgeGroup'] = pd.cut(df2['Age'], [0, 20, 40, 60, 80, np.inf], labels = [1,2,3,4,5], right = True)

display_side_by_side(df,df2)

ID 
 Names 
 Age 
 Values 
 
 
 
 
 0 
 1 
 Kate 
 39 
 3 
 
 
 1 
 2 
 John 
 18 
 4 
 
 
 2 
 3 
 Max 
 25 
 8 
 
 
 3 
 4 
 Kate 
 75 
 12 
 
 
 
 
 
 
 ID 
 Names 
 Age 
 Values 
 ValGroup1 
 ValGroup2 
 AgeGroup 
 
 
 
 
 0 
 1 
 Kate 
 39 
 3 
 (2.991, 5.25] 
 (2.999, 3.75] 
 2 
 
 
 1 
 2 
 John 
 18 
 4 
 (2.991, 5.25] 
 (3.75, 6.0] 
 1 
 
 
 2 
 3 
 Max 
 25 
 8 
 (7.5, 9.75] 
 (6.0, 9.0] 
 2 
 
 
 3 
 4 
 Kate 
 75 
 12 
 (9.75, 12.0] 
 (9.0, 12.0] 
 4

### Variable to dummy columns

Here type Categorical but applies to other types as well.

In [49]:
df = pd.DataFrame({'Category': ['Type1','Type2','Type1','Type3'], 'Value': np.random.rand(4)})

df2 = df.copy()
cats = ['Type1','Type2','Type3']
cat_type = CategoricalDtype(categories=cats, ordered=True)
df['Category'] = df['Category'].astype(cat_type)
df2 = pd.get_dummies(df2, prefix=['Category'])

display_side_by_side(df, df2)

Category 
 Value 
 
 
 
 
 0 
 Type1 
 0.203393 
 
 
 1 
 Type2 
 0.214765 
 
 
 2 
 Type1 
 0.326027 
 
 
 3 
 Type3 
 0.722578 
 
 
 
 
 
 
 Value 
 Category_Type1 
 Category_Type2 
 Category_Type3 
 
 
 
 
 0 
 0.203393 
 1 
 0 
 0 
 
 
 1 
 0.214765 
 0 
 1 
 0 
 
 
 2 
 0.326027 
 1 
 0 
 0 
 
 
 3 
 0.722578 
 0 
 0 
 1

### Dummy columns back to index variable

In [57]:
df = pd.DataFrame({'Category':['Type1','Type2','Type1','Type3'], 'Value':np.random.rand(4)})
df = pd.get_dummies(df, prefix=['Category'])
df["Category_reversed"] = df[["Category_Type1", "Category_Type2", "Category_Type3"]].idxmax(axis=1)
display(df)

,Value,Category_Type1,Category_Type2,Category_Type3,Category_reversed
0,0.372868,1,0,0,Category_Type1
1,0.085579,0,1,0,Category_Type2
2,0.615656,1,0,0,Category_Type1
3,0.328626,0,0,1,Category_Type3


## Change between long/wide format

### Wide to long

In [58]:
# Create data
df = pd.DataFrame(data = np.arange(9).reshape((3,3)), columns = [['col1','col2','col3']])

df2 = pd.DataFrame(data = np.arange(12).reshape((4,3))
                  ,index = [['a','a','b','b',], [1,2] * 2]
                  ,columns = [['col1','col1','col2'], ['ff','gg','ff']])

# Simple melt
df3 = pd.melt(df).copy()
df4 = pd.melt(df2).copy()

display_side_by_side(df,df3,df2,df4)

col1 
 col2 
 col3 
 
 
 
 
 0 
 0 
 1 
 2 
 
 
 1 
 3 
 4 
 5 
 
 
 2 
 6 
 7 
 8 
 
 
 
 
 
 
 None 
 value 
 
 
 
 
 0 
 col1 
 0 
 
 
 1 
 col1 
 3 
 
 
 2 
 col1 
 6 
 
 
 3 
 col2 
 1 
 
 
 4 
 col2 
 4 
 
 
 5 
 col2 
 7 
 
 
 6 
 col3 
 2 
 
 
 7 
 col3 
 5 
 
 
 8 
 col3 
 8 
 
 
 
 
 
 
 
 col1 
 col2 
 
 
 
 
 ff 
 gg 
 ff 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 
 
 2 
 3 
 4 
 5 
 
 
 b 
 1 
 6 
 7 
 8 
 
 
 2 
 9 
 10 
 11 
 
 
 
 
 
 
 variable_0 
 variable_1 
 value 
 
 
 
 
 0 
 col1 
 ff 
 0 
 
 
 1 
 col1 
 ff 
 3 
 
 
 2 
 col1 
 ff 
 6 
 
 
 3 
 col1 
 ff 
 9 
 
 
 4 
 col1 
 gg 
 1 
 
 
 5 
 col1 
 gg 
 4 
 
 
 6 
 col1 
 gg 
 7 
 
 
 7 
 col1 
 gg 
 10 
 
 
 8 
 col2 
 ff 
 2 
 
 
 9 
 col2 
 ff 
 5 
 
 
 10 
 col2 
 ff 
 8 
 
 
 11 
 col2 
 ff 
 11

### Long to wide

In [59]:
df = pd.DataFrame({
         'name1': ['A','A','B','B']
        ,'name2': ['A','B','A','B']
        ,'name3': ['r','t','r','b']    
        ,'val': [5,9,7,3]})

df2 = df.pivot_table(values = 'val', index = ['name1','name3'], columns = 'name2').reset_index()
df2.columns.name = None

display_side_by_side(df,df2)
df2.columns

,name1,name2,name3,val
0,A,A,r,5
1,A,B,t,9
2,B,A,r,7
3,B,B,b,3
,name1,name3,A,B
0,A,r,5.0,NaN
1,A,t,NaN,9.0
2,B,b,NaN,3.0
3,B,r,7.0,NaN


Index(['name1', 'name3', 'A', 'B'], dtype='object')

In [60]:
# Pivot_table might not work with non-numeric columns
# In this case try the following (does however not work
# with duplicate indices

df = pd.DataFrame({'index' : range(8)
                  ,'variable1' : ["A","A","B","B","A","B","B","A"]
                  ,'variable2' : ["a","b","a","b","a","b","a","b"]
                  ,'result': ["on","off","off","on","on","off","off","on"]})

df1 = df.copy()
unstack_cols = ['variable1', 'variable2']
df1 = df.set_index(['index'] + unstack_cols).unstack(level = unstack_cols)

display_side_by_side(df,df1)

index 
 variable1 
 variable2 
 result 
 
 
 
 
 0 
 0 
 A 
 a 
 on 
 
 
 1 
 1 
 A 
 b 
 off 
 
 
 2 
 2 
 B 
 a 
 off 
 
 
 3 
 3 
 B 
 b 
 on 
 
 
 4 
 4 
 A 
 a 
 on 
 
 
 5 
 5 
 B 
 b 
 off 
 
 
 6 
 6 
 B 
 a 
 off 
 
 
 7 
 7 
 A 
 b 
 on 
 
 
 
 
 
 
 result 
 
 
 variable1 
 A 
 B 
 
 
 variable2 
 a 
 b 
 a 
 b 
 
 
 index 
 
 
 
 
 
 
 
 
 0 
 on 
 NaN 
 NaN 
 NaN 
 
 
 1 
 NaN 
 off 
 NaN 
 NaN 
 
 
 2 
 NaN 
 NaN 
 off 
 NaN 
 
 
 3 
 NaN 
 NaN 
 NaN 
 on 
 
 
 4 
 on 
 NaN 
 NaN 
 NaN 
 
 
 5 
 NaN 
 NaN 
 NaN 
 off 
 
 
 6 
 NaN 
 NaN 
 off 
 NaN 
 
 
 7 
 NaN 
 on 
 NaN 
 NaN

In [61]:
# Long to wide with time series type of data
data = { 'ID': [1, 1, 2, 2, 1, 1, 2, 2]
        ,'Date': ['2007-01-01','2007-01-01', '2007-01-01','2007-01-01', '2007-01-02','2007-01-02', '2007-01-02','2007-01-02']
        ,'Type': ['type1','type2','type1','type2','type1','type2','type1','type2']
        ,'Values': np.arange(8)}
df = pd.DataFrame(data = data)

df4 = df.pivot_table(values = 'Values', index = ['Date'], columns = ['ID','Type'])
df4.columns.names = [None,None]
df4.index.name = None
new_col_names = ['_'.join((str(b),str(a))) for a,b in df4.columns.get_values()] # flatten columns
df4.columns = new_col_names

# Below works as well, but if data contains NAs, these are converted to 0 which is not preferable!

#df2 = df.groupby(['Date','ID','Type']).sum().unstack('ID').copy()
#new_col_names = ['_'.join((str(b),a)) for a,b in df2.columns.get_values()] # flatten columns
#df2.columns = new_col_names

#df3 = df.groupby(['Date','ID','Type']).sum().unstack(['ID','Type']).copy()
#new_col_names = ['_'.join((a,str(b),c)) for a,b,c in df3.columns.get_values()]
#df3.columns = new_col_names

display_side_by_side(df,df4)

,ID,Date,Type,Values
0,1,2007-01-01,type1,0
1,1,2007-01-01,type2,1
2,2,2007-01-01,type1,2
3,2,2007-01-01,type2,3
4,1,2007-01-02,type1,4
5,1,2007-01-02,type2,5
6,2,2007-01-02,type1,6
7,2,2007-01-02,type2,7
,type1_1,type2_1,type1_2,type2_2
2007-01-01,0,1,2,3


## Database-like joins

In [62]:
# Create data
df = pd.DataFrame(data = np.arange(12).reshape((3,4)), columns = ['col1','col2','col3','col4'])
df2 = pd.DataFrame({'key': [0,1,2], 'vals': [4,6,8]})
display_side_by_side(df,df2)

col1 
 col2 
 col3 
 col4 
 
 
 
 
 0 
 0 
 1 
 2 
 3 
 
 
 1 
 4 
 5 
 6 
 7 
 
 
 2 
 8 
 9 
 10 
 11 
 
 
 
 
 
 
 key 
 vals 
 
 
 
 
 0 
 0 
 4 
 
 
 1 
 1 
 6 
 
 
 2 
 2 
 8

In [63]:
# Different merges
df3 = pd.merge(df, df2, left_index = True, right_on = 'key', how = 'left')
df4 = pd.merge(df, df2, left_on = 'col2' , right_on = 'key', how = 'left')
df5 = pd.merge(df, df2, left_on = 'col2' , right_on = 'key', how = 'right')
df6 = pd.merge(df, df2, left_on = 'col2' , right_on = 'key', how = 'inner')
df7 = pd.merge(df, df2, left_on = 'col2' , right_on = 'key', how = 'outer')

# Right and left joins are basically the same, just different way around
# Compare df8 to df5
df8 = pd.merge(df2, df, left_on = 'key', right_on = 'col2', how = 'left')

display_side_by_side(df3,df4,df5,df6,df7,df8)

,col1,col2,col3,col4,key,vals
0,0,1,2,3,0,4
1,4,5,6,7,1,6
2,8,9,10,11,2,8
,col1,col2,col3,col4,key,vals
0,0,1,2,3,1.0,6.0
1,4,5,6,7,NaN,NaN
2,8,9,10,11,NaN,NaN
,col1,col2,col3,col4,key,vals
0,0.0,1.0,2.0,3.0,1,6
1,NaN,NaN,NaN,NaN,0,4


In [64]:
# Join on MultiIndex
# DOES NOT WORK FOR SOME REASON... PROLLY DUE TO RangeIndex
df = pd.DataFrame(data = np.arange(12).reshape((4,3))
                  ,index = [['a','a','b','b',], [1,2] * 2]
                  ,columns = [['col1','col1','col2'], ['ff','gg','ff']])
df.index.names = [0,1]

df2 = pd.DataFrame({'key': [0,1,2], 'vals': [4,6,8]})
#df3 = pd.merge(df,df2,left_on = df.index.names[1], right_on = df2.index,how = 'left').set_index(df.index)



display_side_by_side(df,df2)

df.index.names


col1 
 col2 
 
 
 
 
 ff 
 gg 
 ff 
 
 
 0 
 1 
 
 
 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 
 
 2 
 3 
 4 
 5 
 
 
 b 
 1 
 6 
 7 
 8 
 
 
 2 
 9 
 10 
 11 
 
 
 
 
 
 
 key 
 vals 
 
 
 
 
 0 
 0 
 4 
 
 
 1 
 1 
 6 
 
 
 2 
 2 
 8

FrozenList([0, 1])

In [65]:
# Get rows which are both in another DataFrame
df1 = pd.DataFrame(data = {'col1' : [1, 2, 3, 4, 5], 'col2' : [10, 11, 12, 13, 14]}) 
df2 = pd.DataFrame(data = {'col1' : [1, 2, 3], 'col2' : [10, 11, 12]})
df = df1[~df1.isin(df2)].dropna()
display_side_by_side(df1,df2,df)

,col1,col2
0,1,10
1,2,11
2,3,12
3,4,13
4,5,14
,col1,col2
0,1,10
1,2,11
2,3,12
,col1,col2


## Aggregating infromation in DataFrame

### Simple means, medians etc.

In [66]:
# Simple example
df = pd.DataFrame(np.random.rand(4,4), columns = ['ff', 'ee', 'tt', 'uuu'], index = ['first','second','third','fourth'])
df.iloc[:2,1] = np.nan
display(df)

# Mean by column, change axis=0 to get rows
display(df.mean(axis = 1, skipna = False))
display(df.mean(axis = 1, skipna = True))

,ff,ee,tt,uuu
first,0.899523,NaN,0.899533,0.057163
second,0.346913,NaN,0.281189,0.116024
third,0.565758,0.215847,0.128712,0.468037
fourth,0.315411,0.633211,0.190264,0.278555


first          NaN
second         NaN
third     0.344589
fourth    0.354360
dtype: float64

first     0.618740
second    0.248042
third     0.344589
fourth    0.354360
dtype: float64

### Apply means, medians etc. to subportion of data frame

In [67]:
# Easy way to calculate aggregations using lambdas and apply
df = pd.DataFrame(np.random.rand(4,4), columns = ['ff', 'ee', 'tt', 'uuu'], index = ['first','second','third','fourth'])
cols_to_apply = ['ff', 'tt', 'uuu']
df2 = pd.DataFrame(df.apply(lambda x: np.max(x[cols_to_apply]), axis = 1), columns = ['Max of column subset'])
df3 = pd.DataFrame(df.apply(lambda x: np.mean(x[cols_to_apply]), axis = 1), columns = ['Mean of column subset'])

display_side_by_side(df,df2,df3)

ff 
 ee 
 tt 
 uuu 
 
 
 
 
 first 
 0.164332 
 0.797561 
 0.778626 
 0.524901 
 
 
 second 
 0.359374 
 0.683851 
 0.240595 
 0.872700 
 
 
 third 
 0.659622 
 0.178167 
 0.446254 
 0.931536 
 
 
 fourth 
 0.902703 
 0.928785 
 0.980299 
 0.088121 
 
 
 
 
 
 
 Max of column subset 
 
 
 
 
 first 
 0.778626 
 
 
 second 
 0.872700 
 
 
 third 
 0.931536 
 
 
 fourth 
 0.980299 
 
 
 
 
 
 
 Mean of column subset 
 
 
 
 
 first 
 0.489286 
 
 
 second 
 0.490890 
 
 
 third 
 0.679137 
 
 
 fourth 
 0.657041

### 'Group by' in various forms

In [70]:
data = {'ID': [1, 1, 2], 'Names': ['Kate', 'John', 'Max'], 'Age': [50, 25, 40], 'Values': [3, 4, 8]}
df = pd.DataFrame(data = data, index = ['first','second','first'])
df.index.name = 'Index'

result1 = df.groupby(['ID'])['Values'].sum()
result1 = pd.DataFrame(result1)

result2 = df.groupby([df.index])['Age'].mean()
result2 = pd.DataFrame(result2)

result3 = df.groupby([df.index])['Age'].count()
result3 = pd.DataFrame(result3)

display_side_by_side(df,result1,result2,result3)

ID 
 Names 
 Age 
 Values 
 
 
 Index 
 
 
 
 
 
 
 
 
 first 
 1 
 Kate 
 50 
 3 
 
 
 second 
 1 
 John 
 25 
 4 
 
 
 first 
 2 
 Max 
 40 
 8 
 
 
 
 
 
 
 Values 
 
 
 ID 
 
 
 
 
 
 1 
 7 
 
 
 2 
 8 
 
 
 
 
 
 
 Age 
 
 
 Index 
 
 
 
 
 
 first 
 45 
 
 
 second 
 25 
 
 
 
 
 
 
 Age 
 
 
 Index 
 
 
 
 
 
 first 
 2 
 
 
 second 
 1

#### Table of counts/means for categorical variable

In [71]:
data = {'ID': [1, 1, 1, 2, 2], 'Type': ['moo', 'boo', 'boo', np.nan, 'moo'], 'Values': [3, 4, 6, 7, 4]}
df = pd.DataFrame(data = data)

# They magic touch is reset_index: count calculation is done on this new column!
df2 = df[['ID', 'Type']].reset_index().groupby(['ID','Type'], as_index = False).count()
df2.columns = ['ID', 'Type', 'count']

# Carefull with missing values! This works with them and assign them as -1
df3 = df[['ID', 'Type']].fillna(-1).reset_index().groupby(['ID','Type'], as_index = False).count()
df3.columns = ['ID', 'Type', 'count']

df4 = df[['ID','Type','Values']].copy().groupby(['ID','Type'], as_index = False).mean()
df4.columns = ['ID', 'Type', 'mean']

display_side_by_side(df,df2,df3,df4)

,ID,Type,Values
0,1,moo,3
1,1,boo,4
2,1,boo,6
3,2,NaN,7
4,2,moo,4
,ID,Type,count
0,1,boo,2
1,1,moo,1
2,2,moo,1
,ID,Type,count


#### Table of counts for continuous variable

In [72]:
def continuous_binning(pdseries, bins=''):
    '''
    Bins continious variable. Treats also NaNs
    Returns data frame.
    Arguments: 
        - pdseries: pandas.Series to be binned
        - bins (optional): user-defined bins. If
          not supplied then 5+1 bins by default.
    '''
    df = pd.DataFrame()
    if bins == '':
        df['bins'] = pd.cut(pdseries, 5, right = True)        
    else:
        df['bins'] = pd.cut(pdseries, bins, right = True)
    df = df[['bins']].reset_index().groupby(['bins'], as_index = False).count()
    df.columns = ['bins','count']
    df_nans = pd.DataFrame({'bins': ['NaN'], 'count': [len(pdseries[np.isnan(pdseries)])]})
    df = df.append(df_nans, ignore_index = True)
    if df['count'].sum() != len(pdseries):
        raise ValueError('Something wrong in function continuous_binning!')
    return df, bins

df = pd.DataFrame(data = {'ID': [1, 2, 3,4, 5], 'Values': [3.5, 4, 8.7, 12.6, 7]})
df2 = pd.DataFrame(data = {'ID': [1, 2, 3,4, 5], 'Values': [3.5, 4, 8.7, 12.6, np.nan]})
bins = [0, 3, 6, 9, 12, 15, np.inf]

y = df['Values']
df3,_ = continuous_binning(y, bins)

y = df2['Values']
df4,_ = continuous_binning(y, bins)

display_side_by_side(df,df3,df4)

,ID,Values
0,1,3.5
1,2,4.0
2,3,8.7
3,4,12.6
4,5,7.0
,bins,count
0,"(0.0, 3.0]",0
1,"(3.0, 6.0]",2
2,"(6.0, 9.0]",2
3,"(9.0, 12.0]",0


In [74]:
# Select sum of first greatest N values within group
data = {'ID': [1, 2, 1, 2, 1], 'Values': [3, 9, 8, 7, 3]}
df = pd.DataFrame(data = data)

df2 = df.copy()
df2.sort_values(['ID','Values'], ascending = [True, False], inplace = True)
df2 = df2.groupby('ID').head(2)
df2 = pd.DataFrame(df2.groupby(['ID'])['Values'].sum().sort_values(ascending = False))

display_side_by_side(df,df2)

ID 
 Values 
 
 
 
 
 0 
 1 
 3 
 
 
 1 
 2 
 9 
 
 
 2 
 1 
 8 
 
 
 3 
 2 
 7 
 
 
 4 
 1 
 3 
 
 
 
 
 
 
 Values 
 
 
 ID 
 
 
 
 
 
 2 
 16 
 
 
 1 
 11

In [75]:
# Combination of group by and apply: caluclation between grouped columns
# Example: Total price of each product bought

df = pd.DataFrame({'Product': [1, 2, 1, 2], 'Quantity': [1, 3, 2, 2], 'Price': [10, 20, 5, 2]})
df2 = pd.DataFrame(df.groupby(['Product']).apply(lambda df, a, b: sum(df[a] * df[b]), 'Quantity', 'Price'))
df2.columns = ['Total sell price']
display_side_by_side(df,df2)

Product 
 Quantity 
 Price 
 
 
 
 
 0 
 1 
 1 
 10 
 
 
 1 
 2 
 3 
 20 
 
 
 2 
 1 
 2 
 5 
 
 
 3 
 2 
 2 
 2 
 
 
 
 
 
 
 Total sell price 
 
 
 Product 
 
 
 
 
 
 1 
 20 
 
 
 2 
 64

#### Weighted average with group by

In [76]:
df = pd.DataFrame({"name":["Foo", "Baar", "Foo", "Baar"]
                   ,"score_1":[5,10,15,10]
                   ,"score_2" :[10,15,10,25]
                   ,"score_3" : [10,20,30,40]})
df = pd.DataFrame(df.groupby(['name','score_1']).apply(lambda x: np.average(x['score_2'], weights=x['score_3'])))
display(df)

0
name score_1           
Baar 10       21.666667
Foo  5        10.000000
     15       10.000000

#### SQL-like group by - sum combo

In [77]:
df = pd.DataFrame({'Product': [1, 2, 1, 2], 'Quantity': [1, 3, 2, 2], 'Price': [10, 20, 5, 2]})
df['counts_area'] = df['Price'].groupby(df['Product']).transform('sum')
display(df)

,Product,Quantity,Price,counts_area
0,1,1,10,15
1,2,3,20,22
2,1,2,5,15
3,2,2,2,22


### Agg method

In [81]:
# Helper function
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

# Agg is similar to apply, but agg gives the flexibility of
# applying multiple functions at once. Agg() is handy at handling
# DataFrameGroupBy objects compared to apply()

df = pd.DataFrame({"name":["Foo", "Baar", "Foo", "Baar"]
                   ,"score_1":[1,2,1,4]
                   ,"score_2" :[5,10,1,10]
                   ,"score_3" : [5,10,10,10]})

# Mean and sum of "score_2" grouped by "name" and "score_1" columns using apply
print(df.groupby(["name", "score_1"])["score_2"].apply(lambda x : x.mean()))
print(df.groupby(["name", "score_1"])["score_2"].apply(lambda x : x.sum()))

# Same with agg(), and even more aggregations applied to multiple columns, all at the same go!
df2 = df.groupby(["name", "score_1"]).agg({"score_2" :[np.mean, np.sum, np.max, np.min], "score_3": [np.mean]}).copy()


# THIS IS PRIME EXAMPLE ON HOW TO CALCULATE SIMULTANEOUSLY CONDITIONAL DESCRIPTIVE STATISTICS!
# You can also pass in a lambd/customa functions to agg
# https://stackoverflow.com/questions/31521027/groupby-weighted-average-and-sum-in-pandas-dataframe
f_wmean = lambda x: np.average(x, weights=df.loc[x.index, "score_2"])
f_mean = lambda x: np.average(x)
f = {'score_2': ['sum','count', f_mean, percentile(50)],  'score_3': {'mean_w' : f_wmean, 'mean' : f_mean, 'count': 'count'} }
df3 = df.groupby(["name", "score_1"]).agg(f).copy()

display_side_by_side(df,df3)

name  score_1
Baar  2          10.0
      4          10.0
Foo   1           3.0
Name: score_2, dtype: float64
name  score_1
Baar  2          10
      4          10
Foo   1           6
Name: score_2, dtype: int64


name 
 score_1 
 score_2 
 score_3 
 
 
 
 
 0 
 Foo 
 1 
 5 
 5 
 
 
 1 
 Baar 
 2 
 10 
 10 
 
 
 2 
 Foo 
 1 
 1 
 10 
 
 
 3 
 Baar 
 4 
 10 
 10 
 
 
 
 
 
 
 
 score_2 
 score_3 
 
 
 
 
 sum 
 count 
 <lambda_0> 
 percentile_50 
 mean_w 
 mean 
 count 
 
 
 name 
 score_1 
 
 
 
 
 
 
 
 
 
 
 
 Baar 
 2 
 10 
 1 
 10 
 10 
 10.000000 
 10.0 
 1 
 
 
 4 
 10 
 1 
 10 
 10 
 10.000000 
 10.0 
 1 
 
 
 Foo 
 1 
 6 
 2 
 3 
 3 
 5.833333 
 7.5 
 2

#### Corollary to agg method: flatten multi-indexed frame for later use

In [82]:
new_col_names = ['_'.join((str(b),a)) for a,b in df3.columns.get_values()] # flatten columns
#new_col_names = [put names here] # or specify own names
df3.columns = new_col_names
df3.reset_index(inplace=True) # this resets multi index to columns
display(df3)

,name,score_1,sum_score_2,count_score_2,<lambda_0>_score_2,percentile_50_score_2,mean_w_score_3,mean_score_3,count_score_3
0,Baar,2,10,1,10,10,10.000000,10.0,1
1,Baar,4,10,1,10,10,10.000000,10.0,1
2,Foo,1,6,2,3,3,5.833333,7.5,2


### Pivot table

In [83]:
df = pd.DataFrame({"name":["Foo", "Baar", "Foo", "Baar"]
                   ,"name2":['mm','mm','mm','ee']
                   ,"score_1" :[10,15,10,25]
                   ,"score_2" : [10,20,30,40]})

# Basics
df2 = df.pivot_table(values='score_1', index='name', columns = 'name2', aggfunc = [np.mean,np.max]).copy()
df3 = df.pivot_table(values='score_1', index='name', columns = 'name2', aggfunc = [np.mean,np.min], margins = True).copy()

# Shares out of row sum
df4 = df.pivot_table(values = 'score_1', index = 'name', columns = 'name2', aggfunc = [np.sum], margins = True).copy()
df4.columns = df4.columns.get_level_values(1)
df4.drop(['All'], axis = 0 , inplace = True)
df4.fillna(0, inplace = True)
df5 = df4.copy()
for _, row in df5.iterrows():
    row[0:-1] = row[0:-1] / row[-1]

display_side_by_side(df,df2,df3)
display_side_by_side(df4, df5)

name 
 name2 
 score_1 
 score_2 
 
 
 
 
 0 
 Foo 
 mm 
 10 
 10 
 
 
 1 
 Baar 
 mm 
 15 
 20 
 
 
 2 
 Foo 
 mm 
 10 
 30 
 
 
 3 
 Baar 
 ee 
 25 
 40 
 
 
 
 
 
 
 mean 
 amax 
 
 
 name2 
 ee 
 mm 
 ee 
 mm 
 
 
 name 
 
 
 
 
 
 
 
 
 Baar 
 25.0 
 15.0 
 25.0 
 15.0 
 
 
 Foo 
 NaN 
 10.0 
 NaN 
 10.0 
 
 
 
 
 
 
 mean 
 amin 
 
 
 name2 
 ee 
 mm 
 All 
 ee 
 mm 
 All 
 
 
 name 
 
 
 
 
 
 
 
 
 
 
 Baar 
 25.0 
 15.000000 
 20 
 25.0 
 15.0 
 15 
 
 
 Foo 
 NaN 
 10.000000 
 10 
 NaN 
 10.0 
 10 
 
 
 All 
 25.0 
 11.666667 
 15 
 25.0 
 10.0 
 10

name2,ee,mm,All
name,,,
Baar,25.0,15.0,40
Foo,0.0,20.0,20
name2,ee,mm,All
name,,,
Baar,25.0,15.0,40
Foo,0.0,20.0,20


### List observations with count > n

In [84]:
df = pd.DataFrame({"name":["Foo", "Baar", "Foo", "Foo"]
                   ,"name2":['mm','mm','mm','ee']
                   ,"score_1" :[10,15,10,25]
                   ,"score_2" : [10,20,30,40]})

counts = df['name'].value_counts()
counts  = counts[counts > 2].index

df[df['name'].isin(counts)]


,name,name2,score_1,score_2
0,Foo,mm,10,10
2,Foo,mm,10,30
3,Foo,ee,25,40


<h1>Data cleaning</h1>

## Find and replace string in data frame column

In [85]:
data = {'ID': [1, 2, 3], 'Names': ['Kate', 'John', 'Max'], 'Age': [50, 25, 41], 'Values': [3, 4, 8]}
df = pd.DataFrame(data=data, columns = ['ID', 'Names', 'Age', 'Values', 'Country'])

# find
df2 = df[df['Names'].str.contains("John")].copy()

# replace
df3 = df.copy()
df3['Names'] = df3['Names'].str.replace("John",'Mike')

display_side_by_side(df,df2,df3)

,ID,Names,Age,Values,Country
0,1,Kate,50,3,NaN
1,2,John,25,4,NaN
2,3,Max,41,8,NaN
,ID,Names,Age,Values,Country
1,2,John,25,4,NaN
,ID,Names,Age,Values,Country
0,1,Kate,50,3,NaN
1,2,Mike,25,4,NaN
2,3,Max,41,8,NaN


## Replace garbage values with NaN

### Replace non-numerical values in numerical colun with NaN

In [86]:
df = pd.DataFrame(np.random.rand(4,4), columns = ['ff', 'ee', 'tt', 'uuu'], index = ['first','second','third','fourth'])
df.iloc[2,1] = 'dd'
df.iloc[1,2] = '-----'
df.iloc[2,2] = '#SomeErrorVal'
df2 = df.copy()

# Count amount of non-numeric in a column
print('Column ' + 'ee' + ' has {} non-numeric'.format(len(df.loc[pd.to_numeric(df['ee'], errors='coerce').isnull(), 'ee'])))

cols = ['ff', 'ee', 'tt', 'uuu']
for col in cols:
    df2[col][df2.apply(lambda x: np.isreal(x[col]) == True , axis = 1) == False] = np.nan

display_side_by_side(df,df2)

Column ee has 1 non-numeric


,ff,ee,tt,uuu
first,0.575688,0.120615,0.523382,0.418098
second,0.528889,0.346107,-----,0.965802
third,0.199198,dd,#SomeErrorVal,0.469019
fourth,0.040665,0.405705,0.366432,0.837181
,ff,ee,tt,uuu
first,0.575688,0.120615,0.523382,0.418098
second,0.528889,0.346107,NaN,0.965802
third,0.199198,NaN,NaN,0.469019
fourth,0.040665,0.405705,0.366432,0.837181


### Replace non-numerical values in string colun with NaN
Needs regex

In [87]:
data = {'ID': [1, 2, 3], 'Names': ['Kate', 'John', 'Max'], 'Values': ['3', '4', '.']}
df = pd.DataFrame(data=data, columns = ['ID', 'Names', 'Values'])

df2 = df.copy()
df2['Values'] = df2['Values'].replace(r'\.', np.nan, regex=True)

display_side_by_side(df,df2)


,ID,Names,Values
0,1,Kate,3
1,2,John,4
2,3,Max,.
,ID,Names,Values
0,1,Kate,3
1,2,John,4
2,3,Max,NaN


## Find NaN values from DataFrame

In [88]:
df = pd.DataFrame(np.random.rand(4,4), columns = ['ff', 'ee', 'tt', 'uuu'], index = ['first','second','third','fourth'])
df.iloc[2,1] = np.nan
df.iloc[0,0] = np.nan

# Calculate number of NaNs in column
print('Column ' + 'ee' + ' has {} NaNs'.format(pd.isnull(df['ee']).sum()))

df2 = df[df.isnull().any(axis=1)].copy()

display_side_by_side(df, df2)

Column ee has 1 NaNs


,ff,ee,tt,uuu
first,NaN,0.530301,0.107439,0.091845
second,0.300896,0.189447,0.654256,0.971062
third,0.929798,NaN,0.893517,0.253335
fourth,0.153778,0.020400,0.641426,0.522232
,ff,ee,tt,uuu
first,NaN,0.530301,0.107439,0.091845
third,0.929798,NaN,0.893517,0.253335


## Clean NaN values

In [89]:
df = pd.DataFrame(np.random.rand(4,4), columns = ['ff', 'ee', 'tt', 'uuu'], index = ['first','second','third','fourth'])
df.iloc[2,1] = np.nan
df.iloc[1,3] = np.nan
df.iloc[0,0] = np.nan
df2 = df.copy()
df3 = df.copy()

# Fill NaN by mean of the column
df2['ee'].fillna(df['ee'].mean(), inplace = True)

# Fill using value from previuous row
df2['uuu'].fillna(method='bfill', inplace = True)

# Fill using value from next row
df2['ff'].fillna(method='bfill', inplace = True)

# Fill multiple columns by some scalar value
df3[['ee','uuu', 'ff']] = df3[['ee','uuu', 'ff']].fillna(0)

display_side_by_side(df,df2,df3)

,ff,ee,tt,uuu
first,NaN,0.625354,0.774955,0.412072
second,0.705570,0.560440,0.419763,NaN
third,0.390283,NaN,0.592119,0.628432
fourth,0.937464,0.326567,0.511627,0.607489
,ff,ee,tt,uuu
first,0.705570,0.625354,0.774955,0.412072
second,0.705570,0.560440,0.419763,0.628432
third,0.390283,0.504121,0.592119,0.628432
fourth,0.937464,0.326567,0.511627,0.607489
,ff,ee,tt,uuu


## Drop rows/columns with NaN values

In [90]:
df = pd.DataFrame([[np.nan, 2, np.nan, 0], [3, 4, np.nan, 1],
                   [np.nan, np.nan, np.nan, 5],[7, 1, 6, 5]],
                   columns=list('ABCD'))

# Drop rows (axis = 0)/ columns (axis = 1) where all elements are NaN
df2 = df.dropna(axis = 1, how = 'all').copy()

# Drop rows (axis = 0)/ columns (axis = 1) where any element is NaN
df3 = df.dropna(axis = 0, how = 'any').copy()

# Keep rows (axis = 0)/ columns (axis = 1) which have at least thresh non-NaN values
df4 = df.dropna(axis = 1, thresh = 2).copy()

# Drop NAs in given columns with printgin what happens
df5 = df.copy()
for colname in df5.columns:
    print('Dropping {} observations due to NaN in {}'.format(pd.isnull(df5[colname]).sum(), colname))
    df5 = df5[pd.isnull(df5[colname]) == False]

display_side_by_side(df, df2, df3, df4, df5)

Dropping 2 observations due to NaN in A
Dropping 0 observations due to NaN in B
Dropping 1 observations due to NaN in C
Dropping 0 observations due to NaN in D


,A,B,C,D
0,NaN,2.0,NaN,0
1,3.0,4.0,NaN,1
2,NaN,NaN,NaN,5
3,7.0,1.0,6.0,5
,A,B,C,D
0,NaN,2.0,NaN,0
1,3.0,4.0,NaN,1
2,NaN,NaN,NaN,5
3,7.0,1.0,6.0,5
,A,B,C,D


## Drop duplicates
subset : column label or sequence of labels, optional
Only consider certain columns for identifying duplicates, by default use all of the columns

keep : {‘first’, ‘last’, False}, default ‘first’
first : Mark duplicates as True except for the first occurrence.
last : Mark duplicates as True except for the last occurrence.
False : Mark all duplicates as True.

In [91]:
data = {'ID': [1, 2, 3], 'Names': ['Kate', 'John', 'Max'], 'Age': [50, 25, 45], 'Values': [3, 8, 8]}
df = pd.DataFrame(data=data, columns = ['ID', 'Names', 'Age', 'Values', 'Country'])

# Checks all columns for duplicate rows
df.duplicated(subset=None, keep='first')

# Checks subset of columns for duplicates
df.duplicated(subset=['Values'], keep='first')

# Drop duplicates
df2 = df.copy()
df2.drop_duplicates(subset=['Values'], keep='first', inplace=True)

display_side_by_side(df,df2)


,ID,Names,Age,Values,Country
0,1,Kate,50,3,NaN
1,2,John,25,8,NaN
2,3,Max,45,8,NaN
,ID,Names,Age,Values,Country
0,1,Kate,50,3,NaN
1,2,John,25,8,NaN


## Find nonnumeric values from string column

In [92]:
data = {'ID': [1, 2, 3], 'Names': ['Kate', 'John', 'Max'], 'Values': ['3', '4g', np.nan]}
df = pd.DataFrame(data=data, columns = ['ID', 'Names', 'Values'])

# including NaNs
df2 = df[pd.to_numeric(df['Values'], errors='coerce').isnull()].copy()

# Exclude Nans
df3 = pd.DataFrame(df.loc[(pd.to_numeric(df['Values'], errors='coerce').isnull()) & (~pd.isnull(df['Values'])),'Values'])

display_side_by_side(df2, df3)

ID 
 Names 
 Values 
 
 
 
 
 1 
 2 
 John 
 4g 
 
 
 2 
 3 
 Max 
 NaN 
 
 
 
 
 
 
 Values 
 
 
 
 
 1 
 4g

# Working with time and dates

## Time and date data types

Pandas use 4 types of time functionalities
<ol>
  <li>Timestamp: a point in time</li>
  <li>Period: period spanning over time range</li>
  <li>DatetimeIndex</li>
  <li>PeriodIndex</li>
</ol>

<b> It is recommended to use yyyy-mm-dd convention for dates!</b> This avoids the danger of misinterpreting months vs. days.

In [93]:
# Both of these create a TimeStamp, but only latter accepts frontward European convention!
display(pd.Timestamp('2008-12-10'))
display(pd.to_datetime('10-12-2008', dayfirst = True, format = '%d-%m-%Y')) # only to_datime has dayfirst option!

Timestamp('2008-12-10 00:00:00')

Timestamp('2008-12-10 00:00:00')

In [94]:
# Examples of periods
display(pd.Period('2013'))
display(pd.Period('2017Q1'))
display(pd.Period('05/2017'))
display(pd.Period('2017-12-10'))

Period('2013', 'A-DEC')

Period('2017Q1', 'Q-DEC')

Period('2017-05', 'M')

Period('2017-12-10', 'D')

In [95]:
# TimeStamp and Period indices in DataFrames
df1 = pd.DataFrame(np.arange(3), [pd.Timestamp('2008-12-10'), pd.Timestamp('2008-12-11'), pd.Timestamp('2008-12-12')])
df2 = pd.DataFrame(np.arange(3), [pd.Period('2008-12-10'), pd.Period('2008-12-11'), pd.Period('2008-12-12')])
df3 = pd.DataFrame(np.arange(3), [pd.Period('2008Q2'), pd.Period('2008Q3'), pd.Period('2008Q4')])
df4 = pd.DataFrame(np.arange(4), pd.date_range('2016-10-01', periods=4, freq='1D'))
df5 = pd.DataFrame(np.arange(4), pd.date_range('2016-10-01', periods=4, freq='3W-SUN')) #SUN specifies we want only sundays

display_side_by_side(df1,df2,df3,df4,df5)

,0
2008-12-10,0
2008-12-11,1
2008-12-12,2
,0
2008-12-10,0
2008-12-11,1
2008-12-12,2
,0
2008Q2,0
2008Q3,1


In [96]:
# Convert column into a PeriodIndex
df = pd.DataFrame({'Quarter': ['2008-01-01','2008-04-01','2008-07-01'], 'Values': [5,3,8]})
df2 = pd.DataFrame()
df2['Values'] = df['Values']
df2.index = pd.PeriodIndex(pd.to_datetime(df['Quarter']), freq = '1Q')
df2.index.name = None

df3 = pd.DataFrame({'Year': ['2008', '2009', '2010'], 'Values': [5,3,8]})
df4 = pd.DataFrame()
df4['Values'] = df3['Values']
df4.index = pd.PeriodIndex(pd.to_datetime(df3['Year']), freq = '1A')
df4.index.name = None

display_side_by_side(df,df2,df3,df4)

Quarter 
 Values 
 
 
 
 
 0 
 2008-01-01 
 5 
 
 
 1 
 2008-04-01 
 3 
 
 
 2 
 2008-07-01 
 8 
 
 
 
 
 
 
 Values 
 
 
 
 
 2008Q1 
 5 
 
 
 2008Q2 
 3 
 
 
 2008Q3 
 8 
 
 
 
 
 
 
 Year 
 Values 
 
 
 
 
 0 
 2008 
 5 
 
 
 1 
 2009 
 3 
 
 
 2 
 2010 
 8 
 
 
 
 
 
 
 Values 
 
 
 
 
 2008 
 5 
 
 
 2009 
 3 
 
 
 2010 
 8

In [97]:
#TimeDelta is the difference between two TimeStamps
display(pd.Timestamp('2008-12-10') - pd.Timestamp('2009-04-10'))

Timedelta('-121 days +00:00:00')

### Working with Period and PeriodIndex
These are slightly different creatures and one needs to be careful!

In [98]:
df = pd.DataFrame({'date': ['2008-01-01','2009-01-01','2010-01-01'], 'Values': [5,3,8]})
df.index = pd.PeriodIndex(pd.to_datetime(df['date']), freq = '1A')
df['date_p'] = pd.PeriodIndex(pd.to_datetime(df['date']), freq = '1A')

print('Index is here PeriodIndex but column date_p is Period!')
display(df)

thr = 2009

print("For PeriodIndex stuff is pretty easy")
display(df[df.index < thr])
print(df.index.strftime("%Y"))

print('\n')

print("For Period stuff is harder!")
#df[df['date_p'] < thr] # this causes an error!
display(df[df['date_p'] < pd.Period(thr,'1A')]) # thr needs to be converted to Period as well!
print([x.strftime("%Y") for x in df['date_p'].values])


Index is here PeriodIndex but column date_p is Period!


,date,Values,date_p
date,,,
2008,2008-01-01,5,2008
2009,2009-01-01,3,2009
2010,2010-01-01,8,2010


For PeriodIndex stuff is pretty easy


,date,Values,date_p
date,,,
2008,2008-01-01,5,2008


Index(['2008', '2009', '2010'], dtype='object', name='date')


For Period stuff is harder!


,date,Values,date_p
date,,,
2008,2008-01-01,5,2008


['2008', '2009', '2010']


## Date columns in data frame

### Convert string column to datetime
https://stackoverflow.com/questions/26763344/convert-pandas-column-to-datetime

In [99]:
data = {'value': [1, 2, 3], 'dates': ['01-01-2007', '01-01-2008','01-01-2009']}
df = pd.DataFrame(data=data)

df1 = df.copy()
df1['dates'] =  pd.to_datetime(df['dates'], format='%d-%m-%Y')

display_side_by_side(df,df1)
print("{} vs {}".format(df['dates'].dtype,df1['dates'].dtype))

,value,dates
0,1,01-01-2007
1,2,01-01-2008
2,3,01-01-2009
,value,dates
0,1,2007-01-01
1,2,2008-01-01
2,3,2009-01-01


object vs datetime64[ns]


### Convert datetime to  formatted string

In [100]:
data = {'value': [1, 2, 3], 'dates': pd.to_datetime(['01-01-2007', '01-01-2008','01-01-2009'])}
df = pd.DataFrame(data=data)
df['dates'] = df['dates'].dt.strftime('%Y-%m-%d')
df

,value,dates
0,1,2007-01-01
1,2,2008-01-01
2,3,2009-01-01


### Harmonize dates in data frame column
Sometimes date column might not be harmonized enough to be converted directly to datetime.

In [101]:
'''
Top 2 answers here: https://stackoverflow.com/questions/23581128/how-to-format-date-string-via-multiple-formats-in-python
See also http://strftime.org/
'''
import dateutil.parser
from datetime import datetime

data = {'value': [1, 2, 3, 5, 8], 'dates': ['03/25/93', '04-12-2012', 'Feb. 17, 2005', '6/2006', '2010']}
df1 = pd.DataFrame(data=data)
df2 = df1.copy()

# User-defined function for version 2
def try_parsing_date(text):
    for fmt in ('%Y-%m-%d','%m/%d/%y','%m/%d/%Y','%m-%d-%y','%m-%d-%Y',
                '%d %b %Y','%d %B %Y','%B %d, %Y','%b %d, %Y',
                '%B. %d, %Y','%b %d %Y','%b. %d, %Y','%B %d %Y',
                '%B %Y','%B, %Y','%b, %Y','%b %Y','%m/%Y','%Y','%Y%m'):
        try:
            return datetime.strptime(text, fmt)
        except ValueError:
            pass
    print(text)
    raise ValueError('no valid date format found')

# This is version1 relying on dateutil.parser
# It is pretty good in general case, but might fail on
# very custom dates. Further, fallback treatment for eg. 
# missing days or months is werid; it takes current day/month!
datetimes1 = list()
for index, row in df2.iterrows():
    datetimes1.append(dateutil.parser.parse(row['dates']))
df1['dates_dt'] = datetimes1

# Second version with user-defined function relies on
# datetime.strptime. It is more detailed as different date
# formats can be specified separately. Further, fallback treatment
# of missgin days/months is more convenient
datetimes2 = list()
for index, row in df2.iterrows():
    datetimes2.append(try_parsing_date(row['dates']))
df2['dates_dt'] = datetimes2    


display_side_by_side(df1,df2)

,value,dates,dates_dt
0,1,03/25/93,1993-03-25
1,2,04-12-2012,2012-04-12
2,3,"Feb. 17, 2005",2005-02-17
3,5,6/2006,2006-06-19
4,8,2010,2010-01-19
,value,dates,dates_dt
0,1,03/25/93,1993-03-25
1,2,04-12-2012,2012-04-12
2,3,"Feb. 17, 2005",2005-02-17
3,5,6/2006,2006-06-01


## Time series tricks

### Basic operations

In [102]:
# Start indexing from Thursday 2009-10-01
df = pd.DataFrame({'Values': np.random.randn(8).cumsum()}
                  ,index = pd.date_range('2009-10-01'
                  , periods=8, freq='1W-THU'))

# Nth differences
df2 = df.diff(2).copy()

# Downsample to monthly by mean (or sum, min, max)
df3 = df.resample('M').mean().copy()

# Upsample, and upfill
df3b = df.resample('D').ffill().copy()

# Upsample, use interpolation
df3c = df.resample('D').interpolate('linear').copy()

# Slice DataFrame to only include observations between some range
df4 = df['2009-10-09':'2009-11-07'].copy()

# Change frequency
df5 = df.asfreq('2W-THU').copy()

display_side_by_side(df,df2,df3,df3b.head(10),df3c.head(10),df4,df5)

,Values
2009-10-01,-0.313313
2009-10-08,-0.687545
2009-10-15,-0.575132
2009-10-22,-1.676091
2009-10-29,-0.783540
2009-11-05,-0.629293
2009-11-12,0.499819
2009-11-19,-0.349065
,Values
2009-10-01,NaN


<h3>Get mean value per day over multiple years</h3>

In [103]:
from datetime import datetime
from datetime import timedelta

startdate = datetime(2004, 1, 1)
enddate = datetime(2008, 1, 1)
days = (enddate + timedelta(days = 1) - startdate).days
data = np.random.random(days)
dates = [startdate + timedelta(days=x) for x in range(0, days)]

ts = pd.DataFrame(data, dates, columns = ['values'])
ts.index.names = ['Day']

# Average at each day over multiple years
day_averages = ts.groupby([ts.index.month, ts.index.day]).mean()
day_averages.index.names = ['Month', 'Day']
day_averages.columns = ['avg_values']

display_side_by_side(ts.head(10), pd.DataFrame(day_averages).head(10))

,values
Day,
2004-01-01,0.759609
2004-01-02,0.448255
2004-01-03,0.456027
2004-01-04,0.865615
2004-01-05,0.051798
2004-01-06,0.219580
2004-01-07,0.355753
2004-01-08,0.447017
2004-01-09,0.960287


### Lead and lag
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.shift.html

In [104]:
df = pd.DataFrame({'Values': np.random.randn(8).cumsum()}
                  ,index = pd.date_range('2009-10-01'
                  , periods=8, freq='1W-THU'))

df.shift(periods=2)

,Values
2009-10-01,NaN
2009-10-08,NaN
2009-10-15,-0.033678
2009-10-22,-0.954702
2009-10-29,0.080229
2009-11-05,-0.366216
2009-11-12,0.513697
2009-11-19,-0.506282


# Data frame styling

## Global stylings

### Setting number display format

In [105]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## Style frame when displaying

In [106]:
def ColoringIROverSixPercent(val):
    color = 'red' if val > 0.06 else 'black'
    return 'color: %s' % color

In [107]:
data = {'ID': [1, 2, 3],
        'Names':['firm1', 'firm2', 'firm3'],
        'InterestRate': [0.065, 0.11, 0.01],
        'MarketShare': [36, 5, 12],        
        'BalanceSheetTotal': [102000000, 45000, 8035700000]}
df = pd.DataFrame(data=data)

# Display styled data frame
df.style \
.format({
  'BalanceSheetTotal':'{:,.0f}'.format,
  'InterestRate':'{:.1%}'.format, 
  'MarketShare':lambda x: '{:.1%}'.format(x/100),  
}) \
.applymap(ColoringIROverSixPercent, subset=pd.IndexSlice[:,['InterestRate']])

,ID,Names,InterestRate,MarketShare,BalanceSheetTotal
0,1,firm1,6.5%,36.0%,"102,000,000"
1,2,firm2,11.0%,5.0%,"45,000"
2,3,firm3,1.0%,12.0%,"8,035,700,000"


## Style multi-index frame

In [10]:
df = pd.DataFrame(
    data=np.random.uniform(size=[4, 4]),
    index=[['a','a','b','b',], [1,2] * 2],
    columns=[['col1','col1','col2','col2'], ['ff','gg','ff','gg']]
)

formats = {
    "ff":"{:.2f}",
    "gg":"{:.0%}"
}
formats_dict = {
    midx:formats[level_val]
    for level_val in formats
    for midx in [col for col in df if col[1]==level_val]
}
df.style.format(formats_dict)

# Uncategorized

## Re-index columns to start e.g from 100

In [108]:
df = pd.DataFrame({'Values': np.random.randn(8).cumsum()}
                  ,index = pd.date_range('2009-10-01'
                  , periods=8, freq='1W-THU'))
df2 = df.copy()
df2['type'] = 't1'
for col in df2.columns.values[:-1]:
    gg = df2.groupby('type')[col].apply(lambda x: x.div(x.iloc[0]).subtract(1))
    df2[col] = (1 + gg) * 100
df2.drop(['type'], axis = 1, inplace = True)

display_side_by_side(df,df2)

,Values
2009-10-01,-2.231
2009-10-08,-2.030
2009-10-15,-3.090
2009-10-22,-2.597
2009-10-29,-3.395
2009-11-05,-1.191
2009-11-12,-1.538
2009-11-19,-1.649
,Values
2009-10-01,100.000


## Usual data describing stuff

In [109]:
data = {'ID': [1, 2, 3],'Names': ['Kate', 'John', 'Max'], 'Occupation': ['CEO', 'Doctor', 'Driver']
        ,'Vals1': [50, 25, 41], 'Vals2': [3.0, 4.0, 8.0]}
df = pd.DataFrame(data=data)

print("Total length of data frame: {}".format(len(df)))

for name in df.columns:
    print('Column {} has {} non-numeric'.format(name,len(df.loc[pd.to_numeric(df[name], errors='coerce').isnull(), name])))
    print('Column {} has {} NaNs'.format(name,pd.isnull(df[name]).sum()))
    print('-'*15)



Total length of data frame: 3
Column ID has 0 non-numeric
Column ID has 0 NaNs
---------------
Column Names has 3 non-numeric
Column Names has 0 NaNs
---------------
Column Occupation has 3 non-numeric
Column Occupation has 0 NaNs
---------------
Column Vals1 has 0 non-numeric
Column Vals1 has 0 NaNs
---------------
Column Vals2 has 0 non-numeric
Column Vals2 has 0 NaNs
---------------


## Check monotonicity of column

In [110]:
data = {'ID': [1, 2, 3, 4, 5], 'Names': ['Kare', 'John', 'Max', 'Mary', 'Santa'], 'Age': [50, 25, 41,99, 23], 'Values': [3, 4, 8, 4, 2]}
df = pd.DataFrame(data=data)

print(df['ID'].is_monotonic_increasing)
print(df['ID'].is_monotonic_decreasing)
print(df['Age'].is_monotonic_increasing)

True
False
False
